In [1]:
import os
from datetime import datetime
import cv2
from fpdf import FPDF
import re

# 指定视频文件所在的目录路径 (绝对路径)
directory_path = r'D:\download\gene\ppt_real'

# 指定输出图像和PDF的目录 (绝对路径)
output_directory = os.path.join(directory_path, 'output_directory')
os.makedirs(output_directory, exist_ok=True)  # 如果目录不存在，则创建

# 获取目录下所有文件的列表
file_list = os.listdir(directory_path)

# 按照文件的修改时间进行排序
file_list.sort(key=lambda x: os.path.getmtime(os.path.join(directory_path, x)))

# 遍历排序后的文件列表
for file_name in file_list:
    file_path = os.path.join(directory_path, file_name)

    # 检查是否为文件而非子目录
    if not os.path.isfile(file_path):
        continue  # 跳过非文件项

    # 获取文件的修改时间
    modified_time = datetime.fromtimestamp(os.path.getmtime(file_path))
    print(f"正在处理文件: {file_path}")
    print(f"文件名：{file_name}，修改时间：{modified_time}")

    # 打开视频文件
    cap = cv2.VideoCapture(file_path)

    # 检查视频是否成功打开
    if not cap.isOpened():
        print(f"无法打开视频文件: {file_path}")
        continue  # 跳过无法打开的视频文件

    # 初始化变量
    prev_frame_gray_1 = None
    prev_frame_gray_2 = None
    page_images = []
    i = 0

    while True:
        ret, frame = cap.read()

        if not ret:
            break  # 读取完毕或发生错误

        # 检查帧是否为空
        if frame is None:
            continue

        # 确保感兴趣区域的坐标在帧的边界内
        if frame.shape[0] < 720 or frame.shape[1] < 1280:
            print(f"帧尺寸不足，跳过该帧，帧尺寸：{frame.shape}")
            continue

        # 调整感兴趣区域的坐标以适应帧的实际尺寸
        roi_1 = frame[600:680, 1100:1240]  # 感兴趣区域1，左上移动并稍微变大
        roi_2 = frame[280:500, 340:540]    # 感兴趣区域2，扩大一些

        # 在帧上绘制感兴趣区域的矩形框
        cv2.rectangle(frame, (1100, 600), (1240, 680), (0, 255, 0), 2)  # 绘制感兴趣区域1
        cv2.rectangle(frame, (340, 280), (540, 500), (0, 255, 0), 2)    # 绘制感兴趣区域2

        # 将感兴趣区域转换为灰度图像
        roi_gray_1 = cv2.cvtColor(roi_1, cv2.COLOR_BGR2GRAY)
        roi_gray_2 = cv2.cvtColor(roi_2, cv2.COLOR_BGR2GRAY)

        # 初始化前一帧灰度图像
        if prev_frame_gray_1 is None:
            prev_frame_gray_1 = roi_gray_1.copy()
            prev_frame_gray_2 = roi_gray_2.copy()
            continue  # 跳过第一帧的差分检测

        # 计算当前帧和前一帧的差分图像
        frame_diff_1 = cv2.absdiff(roi_gray_1, prev_frame_gray_1)
        frame_diff_2 = cv2.absdiff(roi_gray_2, prev_frame_gray_2)

        # 对差分图像进行阈值处理
        _, threshold_1 = cv2.threshold(frame_diff_1, 30, 255, cv2.THRESH_BINARY)
        _, threshold_2 = cv2.threshold(frame_diff_2, 30, 255, cv2.THRESH_BINARY)

        # 统计阈值图像中非零像素的数量
        diff_pixels_1 = cv2.countNonZero(threshold_1)
        diff_pixels_2 = cv2.countNonZero(threshold_2)

        # 如果两个区域的差分像素数量都超过阈值，则认为帧发生了变化
        if (diff_pixels_1 > 100 and diff_pixels_2 > 300) or diff_pixels_1 > 150 or diff_pixels_2 > 1000:
            print(f"帧发生变化: {i + 1}, 差分像素1: {diff_pixels_1}, 差分像素2: {diff_pixels_2}")

            # 提取文件名中的编号部分
            match = re.search(r'第(\d+)讲', file_name)
            if match:
                lecture_number = match.group(1)
            else:
                lecture_number = 'unknown'

            # 生成简化的文件名
            simplified_file_name = f"{lecture_number}_{i + 1}.jpg"
            page_path = os.path.join(output_directory, simplified_file_name)

            # 尝试保存图像
            try:
                # 检查输出目录是否存在
                if not os.path.exists(output_directory):
                    os.makedirs(output_directory, exist_ok=True)
                    print(f"创建输出目录: {output_directory}")

                # 保存图像
                success = cv2.imwrite(page_path, frame)
                if success:
                    print(f"图像已保存: {page_path}")
                    page_images.append(page_path)
                else:
                    print(f"图像保存失败: {page_path}")
            except Exception as e:
                print(f"保存图像时发生错误: {str(e)}")
            i += 1

        # 更新前一帧
        prev_frame_gray_1 = roi_gray_1.copy()
        prev_frame_gray_2 = roi_gray_2.copy()

        # 显示当前帧 (可选，运行时可关闭窗口加快速度)
        cv2.imshow("Frame", frame)

        # 跳过一定数量的帧以加快处理速度
        current_frame = cap.get(cv2.CAP_PROP_POS_FRAMES)
        max_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        # 计算目标帧的索引（例如跳500帧）
        target_frame = current_frame + 500
        if target_frame >= max_frames:
            break  # 超过总帧数，结束处理
        cap.set(cv2.CAP_PROP_POS_FRAMES, target_frame)

        # 按下 'q' 键退出循环 (可选，已注释)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    # 创建PDF文件
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)

    for image_path in page_images:
        if os.path.exists(image_path):
            pdf.add_page()
            # 计算图片的宽度和高度，以适应PDF页面
            pdf_width = pdf.w - 20  # 左右各10的边距
            pdf.image(image_path, x=10, y=10, w=pdf_width)
        else:
            print(f"图像文件不存在: {image_path}")

    # 保存PDF
    pdf_file_name = os.path.splitext(file_name)[0] + '.pdf'
    pdf_path = os.path.join(output_directory, pdf_file_name)
    try:
        pdf.output(pdf_path)
        print(f"已保存PDF: {pdf_path}")
    except Exception as e:
        print(f"保存PDF时发生错误: {str(e)}")

正在处理文件: D:\download\gene\ppt_real\动植物基因表达调控(研)_于明_动植物基因表达调控(研)(第2讲)_1.mp4
文件名：动植物基因表达调控(研)_于明_动植物基因表达调控(研)(第2讲)_1.mp4，修改时间：2025-01-06 21:10:33.576509
帧发生变化: 1, 差分像素1: 0, 差分像素2: 3837
图像已保存: D:\download\gene\ppt_real\output_directory\2_1.jpg
帧发生变化: 2, 差分像素1: 0, 差分像素2: 2869
图像已保存: D:\download\gene\ppt_real\output_directory\2_2.jpg
帧发生变化: 3, 差分像素1: 0, 差分像素2: 9391
图像已保存: D:\download\gene\ppt_real\output_directory\2_3.jpg
帧发生变化: 4, 差分像素1: 506, 差分像素2: 13745
图像已保存: D:\download\gene\ppt_real\output_directory\2_4.jpg
帧发生变化: 5, 差分像素1: 507, 差分像素2: 8492
图像已保存: D:\download\gene\ppt_real\output_directory\2_5.jpg
帧发生变化: 6, 差分像素1: 0, 差分像素2: 4635
图像已保存: D:\download\gene\ppt_real\output_directory\2_6.jpg
帧发生变化: 7, 差分像素1: 711, 差分像素2: 12816
图像已保存: D:\download\gene\ppt_real\output_directory\2_7.jpg
帧发生变化: 8, 差分像素1: 711, 差分像素2: 16376
图像已保存: D:\download\gene\ppt_real\output_directory\2_8.jpg
帧发生变化: 9, 差分像素1: 0, 差分像素2: 14128
图像已保存: D:\download\gene\ppt_real\output_directory\2_9.jpg
帧发生变化: 10, 差分像素1: 707, 差分像素2

帧发生变化: 35, 差分像素1: 1078, 差分像素2: 24291
图像已保存: D:\download\gene\ppt_real\output_directory\1_35.jpg
帧发生变化: 36, 差分像素1: 1078, 差分像素2: 21924
图像已保存: D:\download\gene\ppt_real\output_directory\1_36.jpg
帧发生变化: 37, 差分像素1: 286, 差分像素2: 20868
图像已保存: D:\download\gene\ppt_real\output_directory\1_37.jpg
帧发生变化: 38, 差分像素1: 286, 差分像素2: 33753
图像已保存: D:\download\gene\ppt_real\output_directory\1_38.jpg
帧发生变化: 39, 差分像素1: 381, 差分像素2: 31983
图像已保存: D:\download\gene\ppt_real\output_directory\1_39.jpg
帧发生变化: 40, 差分像素1: 381, 差分像素2: 16814
图像已保存: D:\download\gene\ppt_real\output_directory\1_40.jpg
帧发生变化: 41, 差分像素1: 0, 差分像素2: 11350
图像已保存: D:\download\gene\ppt_real\output_directory\1_41.jpg
已保存PDF: D:\download\gene\ppt_real\output_directory\动植物基因表达调控(研)_于明_动植物基因表达调控(研)(第1讲)_1.pdf
正在处理文件: D:\download\gene\ppt_real\动植物基因表达调控(研)_于明_动植物基因表达调控(研)(第3讲)_1.mp4
文件名：动植物基因表达调控(研)_于明_动植物基因表达调控(研)(第3讲)_1.mp4，修改时间：2025-01-06 21:11:58.443641
帧发生变化: 1, 差分像素1: 0, 差分像素2: 3754
图像已保存: D:\download\gene\ppt_real\output_directory\3_1.jpg
帧发生变

帧发生变化: 24, 差分像素1: 0, 差分像素2: 15161
图像已保存: D:\download\gene\ppt_real\output_directory\5_24.jpg
帧发生变化: 25, 差分像素1: 0, 差分像素2: 15166
图像已保存: D:\download\gene\ppt_real\output_directory\5_25.jpg
帧发生变化: 26, 差分像素1: 0, 差分像素2: 15163
图像已保存: D:\download\gene\ppt_real\output_directory\5_26.jpg
帧发生变化: 27, 差分像素1: 313, 差分像素2: 42400
图像已保存: D:\download\gene\ppt_real\output_directory\5_27.jpg
帧发生变化: 28, 差分像素1: 313, 差分像素2: 42658
图像已保存: D:\download\gene\ppt_real\output_directory\5_28.jpg
帧发生变化: 29, 差分像素1: 0, 差分像素2: 4813
图像已保存: D:\download\gene\ppt_real\output_directory\5_29.jpg
帧发生变化: 30, 差分像素1: 0, 差分像素2: 6687
图像已保存: D:\download\gene\ppt_real\output_directory\5_30.jpg
帧发生变化: 31, 差分像素1: 0, 差分像素2: 8542
图像已保存: D:\download\gene\ppt_real\output_directory\5_31.jpg
帧发生变化: 32, 差分像素1: 87, 差分像素2: 3393
图像已保存: D:\download\gene\ppt_real\output_directory\5_32.jpg
帧发生变化: 33, 差分像素1: 216, 差分像素2: 4030
图像已保存: D:\download\gene\ppt_real\output_directory\5_33.jpg
帧发生变化: 34, 差分像素1: 215, 差分像素2: 10444
图像已保存: D:\download\gene\ppt_real

帧发生变化: 28, 差分像素1: 17, 差分像素2: 9153
图像已保存: D:\download\gene\ppt_real\output_directory\6_28.jpg
帧发生变化: 29, 差分像素1: 84, 差分像素2: 41591
图像已保存: D:\download\gene\ppt_real\output_directory\6_29.jpg
帧发生变化: 30, 差分像素1: 66, 差分像素2: 40680
图像已保存: D:\download\gene\ppt_real\output_directory\6_30.jpg
帧发生变化: 31, 差分像素1: 0, 差分像素2: 9317
图像已保存: D:\download\gene\ppt_real\output_directory\6_31.jpg
帧发生变化: 32, 差分像素1: 117, 差分像素2: 9549
图像已保存: D:\download\gene\ppt_real\output_directory\6_32.jpg
帧发生变化: 33, 差分像素1: 0, 差分像素2: 9123
图像已保存: D:\download\gene\ppt_real\output_directory\6_33.jpg
帧发生变化: 34, 差分像素1: 362, 差分像素2: 0
图像已保存: D:\download\gene\ppt_real\output_directory\6_34.jpg
帧发生变化: 35, 差分像素1: 362, 差分像素2: 7061
图像已保存: D:\download\gene\ppt_real\output_directory\6_35.jpg
帧发生变化: 36, 差分像素1: 789, 差分像素2: 10056
图像已保存: D:\download\gene\ppt_real\output_directory\6_36.jpg
帧发生变化: 37, 差分像素1: 10444, 差分像素2: 36864
图像已保存: D:\download\gene\ppt_real\output_directory\6_37.jpg
已保存PDF: D:\download\gene\ppt_real\output_directory\动植物基因表达调控(研)_

图像已保存: D:\download\gene\ppt_real\output_directory\9_35.jpg
帧发生变化: 36, 差分像素1: 181, 差分像素2: 10319
图像已保存: D:\download\gene\ppt_real\output_directory\9_36.jpg
帧发生变化: 37, 差分像素1: 181, 差分像素2: 13543
图像已保存: D:\download\gene\ppt_real\output_directory\9_37.jpg
帧发生变化: 38, 差分像素1: 0, 差分像素2: 10594
图像已保存: D:\download\gene\ppt_real\output_directory\9_38.jpg
帧发生变化: 39, 差分像素1: 0, 差分像素2: 14400
图像已保存: D:\download\gene\ppt_real\output_directory\9_39.jpg
帧发生变化: 40, 差分像素1: 0, 差分像素2: 17504
图像已保存: D:\download\gene\ppt_real\output_directory\9_40.jpg
帧发生变化: 41, 差分像素1: 0, 差分像素2: 12433
图像已保存: D:\download\gene\ppt_real\output_directory\9_41.jpg
帧发生变化: 42, 差分像素1: 681, 差分像素2: 14317
图像已保存: D:\download\gene\ppt_real\output_directory\9_42.jpg
帧发生变化: 43, 差分像素1: 1121, 差分像素2: 15548
图像已保存: D:\download\gene\ppt_real\output_directory\9_43.jpg
帧发生变化: 44, 差分像素1: 1025, 差分像素2: 9788
图像已保存: D:\download\gene\ppt_real\output_directory\9_44.jpg
帧发生变化: 45, 差分像素1: 491, 差分像素2: 8845
图像已保存: D:\download\gene\ppt_real\output_directory\9_45.jpg

...

帧发生变化: 7, 差分像素1: 0, 差分像素2: 14996
图像已保存: D:\download\gene\ppt_real\output_directory\21_7.jpg
帧发生变化: 8, 差分像素1: 0, 差分像素2: 6540
图像已保存: D:\download\gene\ppt_real\output_directory\21_8.jpg
帧发生变化: 9, 差分像素1: 332, 差分像素2: 6338
图像已保存: D:\download\gene\ppt_real\output_directory\21_9.jpg
帧发生变化: 10, 差分像素1: 0, 差分像素2: 6020
图像已保存: D:\download\gene\ppt_real\output_directory\21_10.jpg
帧发生变化: 11, 差分像素1: 1159, 差分像素2: 19080
图像已保存: D:\download\gene\ppt_real\output_directory\21_11.jpg
帧发生变化: 12, 差分像素1: 2465, 差分像素2: 10450
图像已保存: D:\download\gene\ppt_real\output_directory\21_12.jpg
帧发生变化: 13, 差分像素1: 2300, 差分像素2: 7605
图像已保存: D:\download\gene\ppt_real\output_directory\21_13.jpg
帧发生变化: 14, 差分像素1: 2776, 差分像素2: 9730
图像已保存: D:\download\gene\ppt_real\output_directory\21_14.jpg
帧发生变化: 15, 差分像素1: 3984, 差分像素2: 13230
图像已保存: D:\download\gene\ppt_real\output_directory\21_15.jpg
帧发生变化: 16, 差分像素1: 1332, 差分像素2: 12813
图像已保存: D:\download\gene\ppt_real\output_directory\21_16.jpg
帧发生变化: 17, 差分像素1: 77, 差分像素2: 10043
图像已保存: D:\downlo

帧发生变化: 44, 差分像素1: 1220, 差分像素2: 6210
图像已保存: D:\download\gene\ppt_real\output_directory\23_44.jpg
帧发生变化: 45, 差分像素1: 1797, 差分像素2: 8551
图像已保存: D:\download\gene\ppt_real\output_directory\23_45.jpg
帧发生变化: 46, 差分像素1: 1777, 差分像素2: 15182
图像已保存: D:\download\gene\ppt_real\output_directory\23_46.jpg
帧发生变化: 47, 差分像素1: 1437, 差分像素2: 16185
图像已保存: D:\download\gene\ppt_real\output_directory\23_47.jpg
帧发生变化: 48, 差分像素1: 822, 差分像素2: 13373
图像已保存: D:\download\gene\ppt_real\output_directory\23_48.jpg
帧发生变化: 49, 差分像素1: 933, 差分像素2: 11700
图像已保存: D:\download\gene\ppt_real\output_directory\23_49.jpg
帧发生变化: 50, 差分像素1: 643, 差分像素2: 5116
图像已保存: D:\download\gene\ppt_real\output_directory\23_50.jpg
帧发生变化: 51, 差分像素1: 507, 差分像素2: 7955
图像已保存: D:\download\gene\ppt_real\output_directory\23_51.jpg
帧发生变化: 52, 差分像素1: 923, 差分像素2: 9531
图像已保存: D:\download\gene\ppt_real\output_directory\23_52.jpg
帧发生变化: 53, 差分像素1: 1195, 差分像素2: 15285
图像已保存: D:\download\gene\ppt_real\output_directory\23_53.jpg
帧发生变化: 54, 差分像素1: 381, 差分像素2: 19415
图像已保

帧发生变化: 31, 差分像素1: 587, 差分像素2: 6877
图像已保存: D:\download\gene\ppt_real\output_directory\25_31.jpg
帧发生变化: 32, 差分像素1: 383, 差分像素2: 10429
图像已保存: D:\download\gene\ppt_real\output_directory\25_32.jpg
帧发生变化: 33, 差分像素1: 0, 差分像素2: 13488
图像已保存: D:\download\gene\ppt_real\output_directory\25_33.jpg
帧发生变化: 34, 差分像素1: 0, 差分像素2: 7101
图像已保存: D:\download\gene\ppt_real\output_directory\25_34.jpg
帧发生变化: 35, 差分像素1: 0, 差分像素2: 2159
图像已保存: D:\download\gene\ppt_real\output_directory\25_35.jpg
帧发生变化: 36, 差分像素1: 483, 差分像素2: 1068
图像已保存: D:\download\gene\ppt_real\output_directory\25_36.jpg
帧发生变化: 37, 差分像素1: 481, 差分像素2: 4197
图像已保存: D:\download\gene\ppt_real\output_directory\25_37.jpg
帧发生变化: 38, 差分像素1: 80, 差分像素2: 6455
图像已保存: D:\download\gene\ppt_real\output_directory\25_38.jpg
帧发生变化: 39, 差分像素1: 80, 差分像素2: 9853
图像已保存: D:\download\gene\ppt_real\output_directory\25_39.jpg
帧发生变化: 40, 差分像素1: 0, 差分像素2: 26196
图像已保存: D:\download\gene\ppt_real\output_directory\25_40.jpg
帧发生变化: 41, 差分像素1: 530, 差分像素2: 0
图像已保存: D:\download\gene\p

帧发生变化: 27, 差分像素1: 0, 差分像素2: 27099
图像已保存: D:\download\gene\ppt_real\output_directory\27_27.jpg
帧发生变化: 28, 差分像素1: 0, 差分像素2: 28270
图像已保存: D:\download\gene\ppt_real\output_directory\27_28.jpg
帧发生变化: 29, 差分像素1: 0, 差分像素2: 1027
图像已保存: D:\download\gene\ppt_real\output_directory\27_29.jpg
帧发生变化: 30, 差分像素1: 0, 差分像素2: 18070
图像已保存: D:\download\gene\ppt_real\output_directory\27_30.jpg
帧发生变化: 31, 差分像素1: 0, 差分像素2: 13236
图像已保存: D:\download\gene\ppt_real\output_directory\27_31.jpg
帧发生变化: 32, 差分像素1: 0, 差分像素2: 11340
图像已保存: D:\download\gene\ppt_real\output_directory\27_32.jpg
帧发生变化: 33, 差分像素1: 0, 差分像素2: 11235
图像已保存: D:\download\gene\ppt_real\output_directory\27_33.jpg
帧发生变化: 34, 差分像素1: 153, 差分像素2: 12267
图像已保存: D:\download\gene\ppt_real\output_directory\27_34.jpg
帧发生变化: 35, 差分像素1: 153, 差分像素2: 21923
图像已保存: D:\download\gene\ppt_real\output_directory\27_35.jpg
帧发生变化: 36, 差分像素1: 153, 差分像素2: 21928
图像已保存: D:\download\gene\ppt_real\output_directory\27_36.jpg
帧发生变化: 37, 差分像素1: 0, 差分像素2: 14527
图像已保存: D:\download\ge

KeyboardInterrupt: 

In [3]:
import os
from datetime import datetime
import cv2
from fpdf import FPDF
import re

# 指定视频文件所在的目录路径 (绝对路径)
directory_path = r'D:\download\gene\ppt_real'

# 指定输出图像和PDF的目录 (绝对路径)
output_directory = os.path.join(directory_path, 'output_directory')
os.makedirs(output_directory, exist_ok=True)  # 如果目录不存在，则创建

# 获取目录下所有文件的列表
file_list = os.listdir(directory_path)

# 按照文件的修改时间进行排序
file_list.sort(key=lambda x: os.path.getmtime(os.path.join(directory_path, x)))

# 遍历排序后的文件列表
for file_name in file_list:
    file_path = os.path.join(directory_path, file_name)

    # 检查是否为文件而非子目录
    if not os.path.isfile(file_path):
        continue  # 跳过非文件项

    # 获取文件的修改时间
    modified_time = datetime.fromtimestamp(os.path.getmtime(file_path))
    print(f"正在处理文件: {file_path}")
    print(f"文件名：{file_name}，修改时间：{modified_time}")

    # 打开视频文件
    cap = cv2.VideoCapture(file_path)

    # 检查视频是否成功打开
    if not cap.isOpened():
        print(f"无法打开视频文件: {file_path}")
        continue  # 跳过无法打开的视频文件

    # 初始化变量
    prev_frame_gray_1 = None
    prev_frame_gray_2 = None
    page_images = []
    i = 0

    while True:
        ret, frame = cap.read()

        if not ret:
            break  # 读取完毕或发生错误

        # 检查帧是否为空
        if frame is None:
            continue

        # 确保感兴趣区域的坐标在帧的边界内
        if frame.shape[0] < 720 or frame.shape[1] < 1280:
            print(f"帧尺寸不足，跳过该帧，帧尺寸：{frame.shape}")
            continue

        # 调整感兴趣区域的坐标以适应帧的实际尺寸
        roi_1 = frame[600:680, 1100:1240]  # 感兴趣区域1，左上移动并稍微变大
        roi_2 = frame[280:500, 340:540]    # 感兴趣区域2，扩大一些

        # 在帧上绘制感兴趣区域的矩形框
        cv2.rectangle(frame, (1100, 600), (1240, 680), (0, 255, 0), 2)  # 绘制感兴趣区域1
        cv2.rectangle(frame, (340, 280), (540, 500), (0, 255, 0), 2)    # 绘制感兴趣区域2

        # 将感兴趣区域转换为灰度图像
        roi_gray_1 = cv2.cvtColor(roi_1, cv2.COLOR_BGR2GRAY)
        roi_gray_2 = cv2.cvtColor(roi_2, cv2.COLOR_BGR2GRAY)

        # 初始化前一帧灰度图像
        if prev_frame_gray_1 is None:
            prev_frame_gray_1 = roi_gray_1.copy()
            prev_frame_gray_2 = roi_gray_2.copy()
            continue  # 跳过第一帧的差分检测

        # 计算当前帧和前一帧的差分图像
        frame_diff_1 = cv2.absdiff(roi_gray_1, prev_frame_gray_1)
        frame_diff_2 = cv2.absdiff(roi_gray_2, prev_frame_gray_2)

        # 对差分图像进行阈值处理
        _, threshold_1 = cv2.threshold(frame_diff_1, 30, 255, cv2.THRESH_BINARY)
        _, threshold_2 = cv2.threshold(frame_diff_2, 30, 255, cv2.THRESH_BINARY)

        # 统计阈值图像中非零像素的数量
        diff_pixels_1 = cv2.countNonZero(threshold_1)
        diff_pixels_2 = cv2.countNonZero(threshold_2)

        # 如果两个区域的差分像素数量都超过阈值，则认为帧发生了变化
        if (diff_pixels_1 > 100 and diff_pixels_2 > 300) or diff_pixels_1 > 150 or diff_pixels_2 > 1000:
            print(f"帧发生变化: {i + 1}, 差分像素1: {diff_pixels_1}, 差分像素2: {diff_pixels_2}")

            # 提取文件名中的编号部分
            match = re.search(r'第(\d+)讲', file_name)
            if match:
                lecture_number = match.group(1)
            else:
                lecture_number = 'unknown'

            # 生成简化的文件名
            simplified_file_name = f"{lecture_number}_{i + 1}.jpg"
            page_path = os.path.join(output_directory, simplified_file_name)

            # 尝试保存图像
            try:
                # 检查输出目录是否存在
                if not os.path.exists(output_directory):
                    os.makedirs(output_directory, exist_ok=True)
                    print(f"创建输出目录: {output_directory}")

                # 保存图像
                success = cv2.imwrite(page_path, frame)
                if success:
                    print(f"图像已保存: {page_path}")
                    page_images.append(page_path)
                else:
                    print(f"图像保存失败: {page_path}")
            except Exception as e:
                print(f"保存图像时发生错误: {str(e)}")
            i += 1

        # 更新前一帧
        prev_frame_gray_1 = roi_gray_1.copy()
        prev_frame_gray_2 = roi_gray_2.copy()

        # 显示当前帧 (可选，运行时可关闭窗口加快速度)
        cv2.imshow("Frame", frame)

        # 跳过一定数量的帧以加快处理速度
        current_frame = cap.get(cv2.CAP_PROP_POS_FRAMES)
        max_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        # 计算目标帧的索引（例如跳500帧）
        target_frame = current_frame + 500
        if target_frame >= max_frames:
            break  # 超过总帧数，结束处理
        cap.set(cv2.CAP_PROP_POS_FRAMES, target_frame)

        # 按下 'q' 键退出循环 (可选，已注释)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    # 创建PDF文件
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)

    for image_path in page_images:
        if os.path.exists(image_path):
            pdf.add_page()
            # 计算图片的宽度和高度，以适应PDF页面
            pdf_width = pdf.w - 20  # 左右各10的边距
            pdf.image(image_path, x=10, y=10, w=pdf_width)
        else:
            print(f"图像文件不存在: {image_path}")

    # 保存PDF
    pdf_file_name = os.path.splitext(file_name)[0] + '.pdf'
    pdf_path = os.path.join(output_directory, pdf_file_name)
    try:
        pdf.output(pdf_path)
        print(f"已保存PDF: {pdf_path}")
    except Exception as e:
        print(f"保存PDF时发生错误: {str(e)}")

正在处理文件: D:\download\gene\ppt_real\动植物基因表达调控(研)_于明_动植物基因表达调控(研)(第2讲)_1.mp4
文件名：动植物基因表达调控(研)_于明_动植物基因表达调控(研)(第2讲)_1.mp4，修改时间：2025-01-06 21:10:33.576509
帧发生变化: 1, 差分像素1: 0, 差分像素2: 3837
图像已保存: D:\download\gene\ppt_real\output_directory\2_1.jpg
帧发生变化: 2, 差分像素1: 0, 差分像素2: 2869
图像已保存: D:\download\gene\ppt_real\output_directory\2_2.jpg
帧发生变化: 3, 差分像素1: 0, 差分像素2: 9391
图像已保存: D:\download\gene\ppt_real\output_directory\2_3.jpg
帧发生变化: 4, 差分像素1: 506, 差分像素2: 13745
图像已保存: D:\download\gene\ppt_real\output_directory\2_4.jpg
帧发生变化: 5, 差分像素1: 507, 差分像素2: 8492
图像已保存: D:\download\gene\ppt_real\output_directory\2_5.jpg
帧发生变化: 6, 差分像素1: 0, 差分像素2: 4635
图像已保存: D:\download\gene\ppt_real\output_directory\2_6.jpg
帧发生变化: 7, 差分像素1: 711, 差分像素2: 12816
图像已保存: D:\download\gene\ppt_real\output_directory\2_7.jpg
帧发生变化: 8, 差分像素1: 711, 差分像素2: 16376
图像已保存: D:\download\gene\ppt_real\output_directory\2_8.jpg
帧发生变化: 9, 差分像素1: 0, 差分像素2: 14128
图像已保存: D:\download\gene\ppt_real\output_directory\2_9.jpg
帧发生变化: 10, 差分像素1: 707, 差分像素2

帧发生变化: 34, 差分像素1: 0, 差分像素2: 5218
图像已保存: D:\download\gene\ppt_real\output_directory\1_34.jpg
帧发生变化: 35, 差分像素1: 1078, 差分像素2: 24291
图像已保存: D:\download\gene\ppt_real\output_directory\1_35.jpg
帧发生变化: 36, 差分像素1: 1078, 差分像素2: 21924
图像已保存: D:\download\gene\ppt_real\output_directory\1_36.jpg
帧发生变化: 37, 差分像素1: 286, 差分像素2: 20868
图像已保存: D:\download\gene\ppt_real\output_directory\1_37.jpg
帧发生变化: 38, 差分像素1: 286, 差分像素2: 33753
图像已保存: D:\download\gene\ppt_real\output_directory\1_38.jpg
帧发生变化: 39, 差分像素1: 381, 差分像素2: 31983
图像已保存: D:\download\gene\ppt_real\output_directory\1_39.jpg
帧发生变化: 40, 差分像素1: 381, 差分像素2: 16814
图像已保存: D:\download\gene\ppt_real\output_directory\1_40.jpg
帧发生变化: 41, 差分像素1: 0, 差分像素2: 11350
图像已保存: D:\download\gene\ppt_real\output_directory\1_41.jpg
已保存PDF: D:\download\gene\ppt_real\output_directory\动植物基因表达调控(研)_于明_动植物基因表达调控(研)(第1讲)_1.pdf
正在处理文件: D:\download\gene\ppt_real\动植物基因表达调控(研)_于明_动植物基因表达调控(研)(第3讲)_1.mp4
文件名：动植物基因表达调控(研)_于明_动植物基因表达调控(研)(第3讲)_1.mp4，修改时间：2025-01-06 21:11:58.443641
帧发

图像已保存: D:\download\gene\ppt_real\output_directory\5_21.jpg
帧发生变化: 22, 差分像素1: 0, 差分像素2: 3596
图像已保存: D:\download\gene\ppt_real\output_directory\5_22.jpg
帧发生变化: 23, 差分像素1: 0, 差分像素2: 4569
图像已保存: D:\download\gene\ppt_real\output_directory\5_23.jpg
帧发生变化: 24, 差分像素1: 0, 差分像素2: 15161
图像已保存: D:\download\gene\ppt_real\output_directory\5_24.jpg
帧发生变化: 25, 差分像素1: 0, 差分像素2: 15166
图像已保存: D:\download\gene\ppt_real\output_directory\5_25.jpg
帧发生变化: 26, 差分像素1: 0, 差分像素2: 15163
图像已保存: D:\download\gene\ppt_real\output_directory\5_26.jpg
帧发生变化: 27, 差分像素1: 313, 差分像素2: 42400
图像已保存: D:\download\gene\ppt_real\output_directory\5_27.jpg
帧发生变化: 28, 差分像素1: 313, 差分像素2: 42658
图像已保存: D:\download\gene\ppt_real\output_directory\5_28.jpg
帧发生变化: 29, 差分像素1: 0, 差分像素2: 4813
图像已保存: D:\download\gene\ppt_real\output_directory\5_29.jpg
帧发生变化: 30, 差分像素1: 0, 差分像素2: 6687
图像已保存: D:\download\gene\ppt_real\output_directory\5_30.jpg
帧发生变化: 31, 差分像素1: 0, 差分像素2: 8542
图像已保存: D:\download\gene\ppt_real\output_directory\5_31.jpg
帧发生变化: 32, 差

KeyboardInterrupt: 

In [4]:
import os
from datetime import datetime
import cv2
from fpdf import FPDF
import re

# 指定视频文件所在的目录路径 (绝对路径)
directory_path = r'D:\download\gene\ppt_real'

# 指定输出图像和PDF的目录 (绝对路径)
output_directory = os.path.join(directory_path, 'output_directory')
os.makedirs(output_directory, exist_ok=True)  # 如果目录不存在，则创建

# 获取目录下所有文件的列表
file_list = os.listdir(directory_path)

# 按照文件的修改时间进行排序
file_list.sort(key=lambda x: os.path.getmtime(os.path.join(directory_path, x)))

# 遍历排序后的文件列表
for file_name in file_list:
    file_path = os.path.join(directory_path, file_name)

    # 检查是否为文件而非子目录
    if not os.path.isfile(file_path):
        continue  # 跳过非文件项

    # 获取文件的修改时间
    modified_time = datetime.fromtimestamp(os.path.getmtime(file_path))
    print(f"正在处理文件: {file_path}")
    print(f"文件名：{file_name}，修改时间：{modified_time}")

    # 打开视频文件
    cap = cv2.VideoCapture(file_path)

    # 检查视频是否成功打开
    if not cap.isOpened():
        print(f"无法打开视频文件: {file_path}")
        continue  # 跳过无法打开的视频文件

    # 初始化变量
    prev_frame_gray_1 = None
    prev_frame_gray_2 = None
    page_images = []
    i = 0

    while True:
        ret, frame = cap.read()

        if not ret:
            break  # 读取完毕或发生错误

        # 检查帧是否为空
        if frame is None:
            continue

        # 确保感兴趣区域的坐标在帧的边界内
        if frame.shape[0] < 720 or frame.shape[1] < 1280:
            print(f"帧尺寸不足，跳过该帧，帧尺寸：{frame.shape}")
            continue

        # 调整感兴趣区域的坐标以适应帧的实际尺寸
        roi_1 = frame[600:680, 1100:1240]  # 感兴趣区域1，左上移动并稍微变大
        roi_2 = frame[280:500, 340:540]    # 感兴趣区域2，扩大一些

        # 在帧上绘制感兴趣区域的矩形框
        cv2.rectangle(frame, (1100, 600), (1240, 680), (0, 255, 0), 2)  # 绘制感兴趣区域1
        cv2.rectangle(frame, (340, 280), (540, 500), (0, 255, 0), 2)    # 绘制感兴趣区域2

        # 将感兴趣区域转换为灰度图像
        roi_gray_1 = cv2.cvtColor(roi_1, cv2.COLOR_BGR2GRAY)
        roi_gray_2 = cv2.cvtColor(roi_2, cv2.COLOR_BGR2GRAY)

        # 初始化前一帧灰度图像
        if prev_frame_gray_1 is None:
            prev_frame_gray_1 = roi_gray_1.copy()
            prev_frame_gray_2 = roi_gray_2.copy()
            continue  # 跳过第一帧的差分检测

        # 计算当前帧和前一帧的差分图像
        frame_diff_1 = cv2.absdiff(roi_gray_1, prev_frame_gray_1)
        frame_diff_2 = cv2.absdiff(roi_gray_2, prev_frame_gray_2)

        # 对差分图像进行阈值处理
        _, threshold_1 = cv2.threshold(frame_diff_1, 30, 255, cv2.THRESH_BINARY)
        _, threshold_2 = cv2.threshold(frame_diff_2, 30, 255, cv2.THRESH_BINARY)

        # 统计阈值图像中非零像素的数量
        diff_pixels_1 = cv2.countNonZero(threshold_1)
        diff_pixels_2 = cv2.countNonZero(threshold_2)

        # 如果两个区域的差分像素数量都超过阈值，则认为帧发生了变化
        if (diff_pixels_1 > 100 and diff_pixels_2 > 300) or diff_pixels_1 > 150 or diff_pixels_2 > 1000:
            print(f"帧发生变化: {i + 1}, 差分像素1: {diff_pixels_1}, 差分像素2: {diff_pixels_2}")

            # 提取文件名中的编号部分
            match = re.search(r'第(\d+)讲', file_name)
            if match:
                lecture_number = match.group(1)
            else:
                lecture_number = 'unknown'

            # 生成简化的文件名
            simplified_file_name = f"{lecture_number}_{i + 1}.jpg"
            page_path = os.path.join(output_directory, simplified_file_name)

            # 尝试保存图像
            try:
                # 检查输出目录是否存在
                if not os.path.exists(output_directory):
                    os.makedirs(output_directory, exist_ok=True)
                    print(f"创建输出目录: {output_directory}")

                # 保存图像
                success = cv2.imwrite(page_path, frame)
                if success:
                    print(f"图像已保存: {page_path}")
                    page_images.append(page_path)
                else:
                    print(f"图像保存失败: {page_path}")
            except Exception as e:
                print(f"保存图像时发生错误: {str(e)}")
            i += 1

        # 更新前一帧
        prev_frame_gray_1 = roi_gray_1.copy()
        prev_frame_gray_2 = roi_gray_2.copy()

        # 显示当前帧 (可选，运行时可关闭窗口加快速度)
        cv2.imshow("Frame", frame)

        # 跳过一定数量的帧以加快处理速度
        current_frame = cap.get(cv2.CAP_PROP_POS_FRAMES)
        max_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        # 计算目标帧的索引（例如跳500帧）
        target_frame = current_frame + 500
        if target_frame >= max_frames:
            break  # 超过总帧数，结束处理
        cap.set(cv2.CAP_PROP_POS_FRAMES, target_frame)

        # 按下 'q' 键退出循环 (可选，已注释)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    # 创建PDF文件
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)

    for image_path in page_images:
        if os.path.exists(image_path):
            pdf.add_page()
            # 计算图片的宽度和高度，以适应PDF页面
            pdf_width = pdf.w - 20  # 左右各10的边距
            pdf.image(image_path, x=10, y=10, w=pdf_width)
        else:
            print(f"图像文件不存在: {image_path}")

    # 保存PDF
    pdf_file_name = os.path.splitext(file_name)[0] + '.pdf'
    pdf_path = os.path.join(output_directory, pdf_file_name)
    try:
        pdf.output(pdf_path)
        print(f"已保存PDF: {pdf_path}")
    except Exception as e:
        print(f"保存PDF时发生错误: {str(e)}")

正在处理文件: D:\download\gene\ppt_real\动植物基因表达调控(研)_于明_动植物基因表达调控(研)(第2讲)_1.mp4
文件名：动植物基因表达调控(研)_于明_动植物基因表达调控(研)(第2讲)_1.mp4，修改时间：2025-01-06 21:10:33.576509
帧发生变化: 1, 差分像素1: 0, 差分像素2: 3837
图像已保存: D:\download\gene\ppt_real\output_directory\2_1.jpg
帧发生变化: 2, 差分像素1: 0, 差分像素2: 2869
图像已保存: D:\download\gene\ppt_real\output_directory\2_2.jpg
帧发生变化: 3, 差分像素1: 0, 差分像素2: 9391
图像已保存: D:\download\gene\ppt_real\output_directory\2_3.jpg
帧发生变化: 4, 差分像素1: 506, 差分像素2: 13745
图像已保存: D:\download\gene\ppt_real\output_directory\2_4.jpg
帧发生变化: 5, 差分像素1: 507, 差分像素2: 8492
图像已保存: D:\download\gene\ppt_real\output_directory\2_5.jpg
帧发生变化: 6, 差分像素1: 0, 差分像素2: 4635
图像已保存: D:\download\gene\ppt_real\output_directory\2_6.jpg
帧发生变化: 7, 差分像素1: 711, 差分像素2: 12816
图像已保存: D:\download\gene\ppt_real\output_directory\2_7.jpg
帧发生变化: 8, 差分像素1: 711, 差分像素2: 16376
图像已保存: D:\download\gene\ppt_real\output_directory\2_8.jpg
帧发生变化: 9, 差分像素1: 0, 差分像素2: 14128
图像已保存: D:\download\gene\ppt_real\output_directory\2_9.jpg
帧发生变化: 10, 差分像素1: 707, 差分像素2

帧发生变化: 34, 差分像素1: 0, 差分像素2: 5218
图像已保存: D:\download\gene\ppt_real\output_directory\1_34.jpg
帧发生变化: 35, 差分像素1: 1078, 差分像素2: 24291
图像已保存: D:\download\gene\ppt_real\output_directory\1_35.jpg
帧发生变化: 36, 差分像素1: 1078, 差分像素2: 21924
图像已保存: D:\download\gene\ppt_real\output_directory\1_36.jpg
帧发生变化: 37, 差分像素1: 286, 差分像素2: 20868
图像已保存: D:\download\gene\ppt_real\output_directory\1_37.jpg
帧发生变化: 38, 差分像素1: 286, 差分像素2: 33753
图像已保存: D:\download\gene\ppt_real\output_directory\1_38.jpg
帧发生变化: 39, 差分像素1: 381, 差分像素2: 31983
图像已保存: D:\download\gene\ppt_real\output_directory\1_39.jpg
帧发生变化: 40, 差分像素1: 381, 差分像素2: 16814
图像已保存: D:\download\gene\ppt_real\output_directory\1_40.jpg
帧发生变化: 41, 差分像素1: 0, 差分像素2: 11350
图像已保存: D:\download\gene\ppt_real\output_directory\1_41.jpg
已保存PDF: D:\download\gene\ppt_real\output_directory\动植物基因表达调控(研)_于明_动植物基因表达调控(研)(第1讲)_1.pdf
正在处理文件: D:\download\gene\ppt_real\动植物基因表达调控(研)_于明_动植物基因表达调控(研)(第3讲)_1.mp4
文件名：动植物基因表达调控(研)_于明_动植物基因表达调控(研)(第3讲)_1.mp4，修改时间：2025-01-06 21:11:58.443641
帧发

帧发生变化: 23, 差分像素1: 0, 差分像素2: 4569
图像已保存: D:\download\gene\ppt_real\output_directory\5_23.jpg
帧发生变化: 24, 差分像素1: 0, 差分像素2: 15161
图像已保存: D:\download\gene\ppt_real\output_directory\5_24.jpg
帧发生变化: 25, 差分像素1: 0, 差分像素2: 15166
图像已保存: D:\download\gene\ppt_real\output_directory\5_25.jpg
帧发生变化: 26, 差分像素1: 0, 差分像素2: 15163
图像已保存: D:\download\gene\ppt_real\output_directory\5_26.jpg
帧发生变化: 27, 差分像素1: 313, 差分像素2: 42400
图像已保存: D:\download\gene\ppt_real\output_directory\5_27.jpg
帧发生变化: 28, 差分像素1: 313, 差分像素2: 42658
图像已保存: D:\download\gene\ppt_real\output_directory\5_28.jpg
帧发生变化: 29, 差分像素1: 0, 差分像素2: 4813
图像已保存: D:\download\gene\ppt_real\output_directory\5_29.jpg
帧发生变化: 30, 差分像素1: 0, 差分像素2: 6687
图像已保存: D:\download\gene\ppt_real\output_directory\5_30.jpg
帧发生变化: 31, 差分像素1: 0, 差分像素2: 8542
图像已保存: D:\download\gene\ppt_real\output_directory\5_31.jpg
帧发生变化: 32, 差分像素1: 87, 差分像素2: 3393
图像已保存: D:\download\gene\ppt_real\output_directory\5_32.jpg
帧发生变化: 33, 差分像素1: 216, 差分像素2: 4030
图像已保存: D:\download\gene\ppt_real\ou

帧发生变化: 28, 差分像素1: 17, 差分像素2: 9153
图像已保存: D:\download\gene\ppt_real\output_directory\6_28.jpg
帧发生变化: 29, 差分像素1: 84, 差分像素2: 41591
图像已保存: D:\download\gene\ppt_real\output_directory\6_29.jpg
帧发生变化: 30, 差分像素1: 66, 差分像素2: 40680
图像已保存: D:\download\gene\ppt_real\output_directory\6_30.jpg
帧发生变化: 31, 差分像素1: 0, 差分像素2: 9317
图像已保存: D:\download\gene\ppt_real\output_directory\6_31.jpg
帧发生变化: 32, 差分像素1: 117, 差分像素2: 9549
图像已保存: D:\download\gene\ppt_real\output_directory\6_32.jpg
帧发生变化: 33, 差分像素1: 0, 差分像素2: 9123
图像已保存: D:\download\gene\ppt_real\output_directory\6_33.jpg
帧发生变化: 34, 差分像素1: 362, 差分像素2: 0
图像已保存: D:\download\gene\ppt_real\output_directory\6_34.jpg
帧发生变化: 35, 差分像素1: 362, 差分像素2: 7061
图像已保存: D:\download\gene\ppt_real\output_directory\6_35.jpg
帧发生变化: 36, 差分像素1: 789, 差分像素2: 10056
图像已保存: D:\download\gene\ppt_real\output_directory\6_36.jpg
帧发生变化: 37, 差分像素1: 10444, 差分像素2: 36864
图像已保存: D:\download\gene\ppt_real\output_directory\6_37.jpg
已保存PDF: D:\download\gene\ppt_real\output_directory\动植物基因表达调控(研)_

帧发生变化: 35, 差分像素1: 0, 差分像素2: 7684
图像已保存: D:\download\gene\ppt_real\output_directory\9_35.jpg
帧发生变化: 36, 差分像素1: 181, 差分像素2: 10319
图像已保存: D:\download\gene\ppt_real\output_directory\9_36.jpg
帧发生变化: 37, 差分像素1: 181, 差分像素2: 13543
图像已保存: D:\download\gene\ppt_real\output_directory\9_37.jpg
帧发生变化: 38, 差分像素1: 0, 差分像素2: 10594
图像已保存: D:\download\gene\ppt_real\output_directory\9_38.jpg
帧发生变化: 39, 差分像素1: 0, 差分像素2: 14400
图像已保存: D:\download\gene\ppt_real\output_directory\9_39.jpg
帧发生变化: 40, 差分像素1: 0, 差分像素2: 17504
图像已保存: D:\download\gene\ppt_real\output_directory\9_40.jpg
帧发生变化: 41, 差分像素1: 0, 差分像素2: 12433
图像已保存: D:\download\gene\ppt_real\output_directory\9_41.jpg
帧发生变化: 42, 差分像素1: 681, 差分像素2: 14317
图像已保存: D:\download\gene\ppt_real\output_directory\9_42.jpg
帧发生变化: 43, 差分像素1: 1121, 差分像素2: 15548
图像已保存: D:\download\gene\ppt_real\output_directory\9_43.jpg
帧发生变化: 44, 差分像素1: 1025, 差分像素2: 9788
图像已保存: D:\download\gene\ppt_real\output_directory\9_44.jpg
帧发生变化: 45, 差分像素1: 491, 差分像素2: 8845
图像已保存: D:\download\gene\p

帧发生变化: 29, 差分像素1: 122, 差分像素2: 4828
图像已保存: D:\download\gene\ppt_real\output_directory\10_29.jpg
帧发生变化: 30, 差分像素1: 0, 差分像素2: 6179
图像已保存: D:\download\gene\ppt_real\output_directory\10_30.jpg
帧发生变化: 31, 差分像素1: 0, 差分像素2: 7717
图像已保存: D:\download\gene\ppt_real\output_directory\10_31.jpg
帧发生变化: 32, 差分像素1: 0, 差分像素2: 1324
图像已保存: D:\download\gene\ppt_real\output_directory\10_32.jpg
帧发生变化: 33, 差分像素1: 0, 差分像素2: 1323
图像已保存: D:\download\gene\ppt_real\output_directory\10_33.jpg
帧发生变化: 34, 差分像素1: 10280, 差分像素2: 11230
图像已保存: D:\download\gene\ppt_real\output_directory\10_34.jpg
帧发生变化: 35, 差分像素1: 10280, 差分像素2: 13084
图像已保存: D:\download\gene\ppt_real\output_directory\10_35.jpg
帧发生变化: 36, 差分像素1: 0, 差分像素2: 8086
图像已保存: D:\download\gene\ppt_real\output_directory\10_36.jpg
帧发生变化: 37, 差分像素1: 0, 差分像素2: 13261
图像已保存: D:\download\gene\ppt_real\output_directory\10_37.jpg
帧发生变化: 38, 差分像素1: 0, 差分像素2: 10946
图像已保存: D:\download\gene\ppt_real\output_directory\10_38.jpg
帧发生变化: 39, 差分像素1: 10385, 差分像素2: 22329
图像已保存: D:\download

帧发生变化: 25, 差分像素1: 0, 差分像素2: 9926
图像已保存: D:\download\gene\ppt_real\output_directory\12_25.jpg
帧发生变化: 26, 差分像素1: 10549, 差分像素2: 34875
图像已保存: D:\download\gene\ppt_real\output_directory\12_26.jpg
已保存PDF: D:\download\gene\ppt_real\output_directory\动植物基因表达调控(研)_于明_动植物基因表达调控(研)(第12讲)_1.pdf
正在处理文件: D:\download\gene\ppt_real\动植物基因表达调控(研)_于明_动植物基因表达调控(研)(第13讲)_1.mp4
文件名：动植物基因表达调控(研)_于明_动植物基因表达调控(研)(第13讲)_1.mp4，修改时间：2025-01-06 21:30:47.623098
帧发生变化: 1, 差分像素1: 0, 差分像素2: 7528
图像已保存: D:\download\gene\ppt_real\output_directory\13_1.jpg
帧发生变化: 2, 差分像素1: 0, 差分像素2: 7055
图像已保存: D:\download\gene\ppt_real\output_directory\13_2.jpg
帧发生变化: 3, 差分像素1: 0, 差分像素2: 3990
图像已保存: D:\download\gene\ppt_real\output_directory\13_3.jpg
帧发生变化: 4, 差分像素1: 0, 差分像素2: 4204
图像已保存: D:\download\gene\ppt_real\output_directory\13_4.jpg
帧发生变化: 5, 差分像素1: 0, 差分像素2: 3919
图像已保存: D:\download\gene\ppt_real\output_directory\13_5.jpg
帧发生变化: 6, 差分像素1: 0, 差分像素2: 3899
图像已保存: D:\download\gene\ppt_real\output_directory\13_6.jpg
帧发生变化: 7, 差分像素1: 0,

帧发生变化: 9, 差分像素1: 0, 差分像素2: 32768
图像已保存: D:\download\gene\ppt_real\output_directory\16_9.jpg
帧发生变化: 10, 差分像素1: 46, 差分像素2: 12965
图像已保存: D:\download\gene\ppt_real\output_directory\16_10.jpg
帧发生变化: 11, 差分像素1: 2156, 差分像素2: 10726
图像已保存: D:\download\gene\ppt_real\output_directory\16_11.jpg
帧发生变化: 12, 差分像素1: 8572, 差分像素2: 11832
图像已保存: D:\download\gene\ppt_real\output_directory\16_12.jpg
帧发生变化: 13, 差分像素1: 7774, 差分像素2: 13675
图像已保存: D:\download\gene\ppt_real\output_directory\16_13.jpg
帧发生变化: 14, 差分像素1: 414, 差分像素2: 12094
图像已保存: D:\download\gene\ppt_real\output_directory\16_14.jpg
帧发生变化: 15, 差分像素1: 0, 差分像素2: 12112
图像已保存: D:\download\gene\ppt_real\output_directory\16_15.jpg
帧发生变化: 16, 差分像素1: 0, 差分像素2: 20695
图像已保存: D:\download\gene\ppt_real\output_directory\16_16.jpg
帧发生变化: 17, 差分像素1: 1748, 差分像素2: 19539
图像已保存: D:\download\gene\ppt_real\output_directory\16_17.jpg
帧发生变化: 18, 差分像素1: 2129, 差分像素2: 6654
图像已保存: D:\download\gene\ppt_real\output_directory\16_18.jpg
帧发生变化: 19, 差分像素1: 4376, 差分像素2: 9984
图像已保存: D:

帧发生变化: 11, 差分像素1: 0, 差分像素2: 3489
图像已保存: D:\download\gene\ppt_real\output_directory\18_11.jpg
帧发生变化: 12, 差分像素1: 0, 差分像素2: 6345
图像已保存: D:\download\gene\ppt_real\output_directory\18_12.jpg
帧发生变化: 13, 差分像素1: 0, 差分像素2: 7475
图像已保存: D:\download\gene\ppt_real\output_directory\18_13.jpg
帧发生变化: 14, 差分像素1: 0, 差分像素2: 8863
图像已保存: D:\download\gene\ppt_real\output_directory\18_14.jpg
帧发生变化: 15, 差分像素1: 0, 差分像素2: 11950
图像已保存: D:\download\gene\ppt_real\output_directory\18_15.jpg
帧发生变化: 16, 差分像素1: 0, 差分像素2: 11110
图像已保存: D:\download\gene\ppt_real\output_directory\18_16.jpg
帧发生变化: 17, 差分像素1: 0, 差分像素2: 10826
图像已保存: D:\download\gene\ppt_real\output_directory\18_17.jpg
帧发生变化: 18, 差分像素1: 0, 差分像素2: 3440
图像已保存: D:\download\gene\ppt_real\output_directory\18_18.jpg
帧发生变化: 19, 差分像素1: 0, 差分像素2: 4022
图像已保存: D:\download\gene\ppt_real\output_directory\18_19.jpg
帧发生变化: 20, 差分像素1: 0, 差分像素2: 7199
图像已保存: D:\download\gene\ppt_real\output_directory\18_20.jpg
帧发生变化: 21, 差分像素1: 0, 差分像素2: 12106
图像已保存: D:\download\gene\ppt_real\

KeyboardInterrupt: 

In [5]:
import os
from datetime import datetime
import cv2
from fpdf import FPDF
import re

# 指定视频文件所在的目录路径 (绝对路径)
directory_path = r'D:\download\gene\ppt_real'

# 指定输出图像和PDF的目录 (绝对路径)
output_directory = os.path.join(directory_path, 'output_directory')
os.makedirs(output_directory, exist_ok=True)  # 如果目录不存在，则创建

# 获取目录下所有文件的列表
file_list = os.listdir(directory_path)

# 按照文件的修改时间进行排序
file_list.sort(key=lambda x: os.path.getmtime(os.path.join(directory_path, x)))

# 遍历排序后的文件列表
for file_name in file_list:
    file_path = os.path.join(directory_path, file_name)

    # 检查是否为文件而非子目录
    if not os.path.isfile(file_path):
        continue  # 跳过非文件项

    # 获取文件的修改时间
    modified_time = datetime.fromtimestamp(os.path.getmtime(file_path))
    print(f"正在处理文件: {file_path}")
    print(f"文件名：{file_name}，修改时间：{modified_time}")

    # 打开视频文件
    cap = cv2.VideoCapture(file_path)

    # 检查视频是否成功打开
    if not cap.isOpened():
        print(f"无法打开视频文件: {file_path}")
        continue  # 跳过无法打开的视频文件

    # 初始化变量
    prev_frame_gray_1 = None
    prev_frame_gray_2 = None
    page_images = []
    i = 0

    while True:
        ret, frame = cap.read()

        if not ret:
            break  # 读取完毕或发生错误

        # 检查帧是否为空
        if frame is None:
            continue

        # 确保感兴趣区域的坐标在帧的边界内
        if frame.shape[0] < 720 or frame.shape[1] < 1280:
            print(f"帧尺寸不足，跳过该帧，帧尺寸：{frame.shape}")
            continue

        # 调整感兴趣区域的坐标以适应帧的实际尺寸
        roi_1 = frame[580:660, 1060:1200]  # 感兴趣区域1，左上移动并稍微变大
        roi_2 = frame[280:500, 340:540]    # 感兴趣区域2，扩大一些

        # 在帧上绘制感兴趣区域的矩形框
        cv2.rectangle(frame, (1060, 580), (1200, 660), (0, 255, 0), 2)  # 绘制感兴趣区域1
        cv2.rectangle(frame, (340, 280), (540, 500), (0, 255, 0), 2)    # 绘制感兴趣区域2

        # 将感兴趣区域转换为灰度图像
        roi_gray_1 = cv2.cvtColor(roi_1, cv2.COLOR_BGR2GRAY)
        roi_gray_2 = cv2.cvtColor(roi_2, cv2.COLOR_BGR2GRAY)

        # 初始化前一帧灰度图像
        if prev_frame_gray_1 is None:
            prev_frame_gray_1 = roi_gray_1.copy()
            prev_frame_gray_2 = roi_gray_2.copy()
            continue  # 跳过第一帧的差分检测

        # 计算当前帧和前一帧的差分图像
        frame_diff_1 = cv2.absdiff(roi_gray_1, prev_frame_gray_1)
        frame_diff_2 = cv2.absdiff(roi_gray_2, prev_frame_gray_2)

        # 对差分图像进行阈值处理
        _, threshold_1 = cv2.threshold(frame_diff_1, 30, 255, cv2.THRESH_BINARY)
        _, threshold_2 = cv2.threshold(frame_diff_2, 30, 255, cv2.THRESH_BINARY)

        # 统计阈值图像中非零像素的数量
        diff_pixels_1 = cv2.countNonZero(threshold_1)
        diff_pixels_2 = cv2.countNonZero(threshold_2)

        # 如果两个区域的差分像素数量都超过阈值，则认为帧发生了变化
        if (diff_pixels_1 > 100 and diff_pixels_2 > 300) or diff_pixels_1 > 150 or diff_pixels_2 > 1000:
            print(f"帧发生变化: {i + 1}, 差分像素1: {diff_pixels_1}, 差分像素2: {diff_pixels_2}")

            # 提取文件名中的编号部分
            match = re.search(r'第(\d+)讲', file_name)
            if match:
                lecture_number = match.group(1)
            else:
                lecture_number = 'unknown'

            # 生成简化的文件名
            simplified_file_name = f"{lecture_number}_{i + 1}.jpg"
            page_path = os.path.join(output_directory, simplified_file_name)

            # 尝试保存图像
            try:
                # 检查输出目录是否存在
                if not os.path.exists(output_directory):
                    os.makedirs(output_directory, exist_ok=True)
                    print(f"创建输出目录: {output_directory}")

                # 保存图像
                success = cv2.imwrite(page_path, frame)
                if success:
                    print(f"图像已保存: {page_path}")
                    page_images.append(page_path)
                else:
                    print(f"图像保存失败: {page_path}")
            except Exception as e:
                print(f"保存图像时发生错误: {str(e)}")
            i += 1

        # 更新前一帧
        prev_frame_gray_1 = roi_gray_1.copy()
        prev_frame_gray_2 = roi_gray_2.copy()

        # 显示当前帧 (可选，运行时可关闭窗口加快速度)
        cv2.imshow("Frame", frame)

        # 跳过一定数量的帧以加快处理速度
        current_frame = cap.get(cv2.CAP_PROP_POS_FRAMES)
        max_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        # 计算目标帧的索引（例如跳500帧）
        target_frame = current_frame + 500
        if target_frame >= max_frames:
            break  # 超过总帧数，结束处理
        cap.set(cv2.CAP_PROP_POS_FRAMES, target_frame)

        # 按下 'q' 键退出循环 (可选，已注释)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    # 创建PDF文件
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)

    for image_path in page_images:
        if os.path.exists(image_path):
            pdf.add_page()
            # 计算图片的宽度和高度，以适应PDF页面
            pdf_width = pdf.w - 20  # 左右各10的边距
            pdf.image(image_path, x=10, y=10, w=pdf_width)
        else:
            print(f"图像文件不存在: {image_path}")

    # 保存PDF
    pdf_file_name = os.path.splitext(file_name)[0] + '.pdf'
    pdf_path = os.path.join(output_directory, pdf_file_name)
    try:
        pdf.output(pdf_path)
        print(f"已保存PDF: {pdf_path}")
    except Exception as e:
        print(f"保存PDF时发生错误: {str(e)}")

正在处理文件: D:\download\gene\ppt_real\动植物基因表达调控(研)_于明_动植物基因表达调控(研)(第2讲)_1.mp4
文件名：动植物基因表达调控(研)_于明_动植物基因表达调控(研)(第2讲)_1.mp4，修改时间：2025-01-06 21:10:33.576509
帧发生变化: 1, 差分像素1: 0, 差分像素2: 3837
图像已保存: D:\download\gene\ppt_real\output_directory\2_1.jpg
帧发生变化: 2, 差分像素1: 0, 差分像素2: 2869
图像已保存: D:\download\gene\ppt_real\output_directory\2_2.jpg
帧发生变化: 3, 差分像素1: 0, 差分像素2: 9391
图像已保存: D:\download\gene\ppt_real\output_directory\2_3.jpg
帧发生变化: 4, 差分像素1: 731, 差分像素2: 13745
图像已保存: D:\download\gene\ppt_real\output_directory\2_4.jpg
帧发生变化: 5, 差分像素1: 732, 差分像素2: 8492
图像已保存: D:\download\gene\ppt_real\output_directory\2_5.jpg
帧发生变化: 6, 差分像素1: 419, 差分像素2: 4635
图像已保存: D:\download\gene\ppt_real\output_directory\2_6.jpg
帧发生变化: 7, 差分像素1: 625, 差分像素2: 12816
图像已保存: D:\download\gene\ppt_real\output_directory\2_7.jpg
帧发生变化: 8, 差分像素1: 206, 差分像素2: 16376
图像已保存: D:\download\gene\ppt_real\output_directory\2_8.jpg
帧发生变化: 9, 差分像素1: 0, 差分像素2: 14128
图像已保存: D:\download\gene\ppt_real\output_directory\2_9.jpg
帧发生变化: 10, 差分像素1: 1334, 差分

帧发生变化: 34, 差分像素1: 0, 差分像素2: 5218
图像已保存: D:\download\gene\ppt_real\output_directory\1_34.jpg
帧发生变化: 35, 差分像素1: 813, 差分像素2: 3
图像已保存: D:\download\gene\ppt_real\output_directory\1_35.jpg
帧发生变化: 36, 差分像素1: 2065, 差分像素2: 24291
图像已保存: D:\download\gene\ppt_real\output_directory\1_36.jpg
帧发生变化: 37, 差分像素1: 1289, 差分像素2: 21924
图像已保存: D:\download\gene\ppt_real\output_directory\1_37.jpg
帧发生变化: 38, 差分像素1: 1790, 差分像素2: 20868
图像已保存: D:\download\gene\ppt_real\output_directory\1_38.jpg
帧发生变化: 39, 差分像素1: 2166, 差分像素2: 33753
图像已保存: D:\download\gene\ppt_real\output_directory\1_39.jpg
帧发生变化: 40, 差分像素1: 797, 差分像素2: 31983
图像已保存: D:\download\gene\ppt_real\output_directory\1_40.jpg
帧发生变化: 41, 差分像素1: 185, 差分像素2: 16814
图像已保存: D:\download\gene\ppt_real\output_directory\1_41.jpg
帧发生变化: 42, 差分像素1: 0, 差分像素2: 11350
图像已保存: D:\download\gene\ppt_real\output_directory\1_42.jpg
已保存PDF: D:\download\gene\ppt_real\output_directory\动植物基因表达调控(研)_于明_动植物基因表达调控(研)(第1讲)_1.pdf
正在处理文件: D:\download\gene\ppt_real\动植物基因表达调控(研)_于明_动植物基因表达调控

帧发生变化: 19, 差分像素1: 0, 差分像素2: 4676
图像已保存: D:\download\gene\ppt_real\output_directory\5_19.jpg
帧发生变化: 20, 差分像素1: 0, 差分像素2: 3924
图像已保存: D:\download\gene\ppt_real\output_directory\5_20.jpg
帧发生变化: 21, 差分像素1: 0, 差分像素2: 1368
图像已保存: D:\download\gene\ppt_real\output_directory\5_21.jpg
帧发生变化: 22, 差分像素1: 3, 差分像素2: 3596
图像已保存: D:\download\gene\ppt_real\output_directory\5_22.jpg
帧发生变化: 23, 差分像素1: 3, 差分像素2: 4569
图像已保存: D:\download\gene\ppt_real\output_directory\5_23.jpg
帧发生变化: 24, 差分像素1: 0, 差分像素2: 15161
图像已保存: D:\download\gene\ppt_real\output_directory\5_24.jpg
帧发生变化: 25, 差分像素1: 0, 差分像素2: 15166
图像已保存: D:\download\gene\ppt_real\output_directory\5_25.jpg
帧发生变化: 26, 差分像素1: 0, 差分像素2: 15163
图像已保存: D:\download\gene\ppt_real\output_directory\5_26.jpg
帧发生变化: 27, 差分像素1: 669, 差分像素2: 42400
图像已保存: D:\download\gene\ppt_real\output_directory\5_27.jpg
帧发生变化: 28, 差分像素1: 668, 差分像素2: 42658
图像已保存: D:\download\gene\ppt_real\output_directory\5_28.jpg
帧发生变化: 29, 差分像素1: 0, 差分像素2: 4813
图像已保存: D:\download\gene\ppt_real\outpu

帧发生变化: 23, 差分像素1: 0, 差分像素2: 33926
图像已保存: D:\download\gene\ppt_real\output_directory\6_23.jpg
帧发生变化: 24, 差分像素1: 0, 差分像素2: 7298
图像已保存: D:\download\gene\ppt_real\output_directory\6_24.jpg
帧发生变化: 25, 差分像素1: 635, 差分像素2: 4316
图像已保存: D:\download\gene\ppt_real\output_directory\6_25.jpg
帧发生变化: 26, 差分像素1: 731, 差分像素2: 1451
图像已保存: D:\download\gene\ppt_real\output_directory\6_26.jpg
帧发生变化: 27, 差分像素1: 96, 差分像素2: 6297
图像已保存: D:\download\gene\ppt_real\output_directory\6_27.jpg
帧发生变化: 28, 差分像素1: 442, 差分像素2: 9153
图像已保存: D:\download\gene\ppt_real\output_directory\6_28.jpg
帧发生变化: 29, 差分像素1: 936, 差分像素2: 41591
图像已保存: D:\download\gene\ppt_real\output_directory\6_29.jpg
帧发生变化: 30, 差分像素1: 530, 差分像素2: 40680
图像已保存: D:\download\gene\ppt_real\output_directory\6_30.jpg
帧发生变化: 31, 差分像素1: 0, 差分像素2: 9317
图像已保存: D:\download\gene\ppt_real\output_directory\6_31.jpg
帧发生变化: 32, 差分像素1: 405, 差分像素2: 8
图像已保存: D:\download\gene\ppt_real\output_directory\6_32.jpg
帧发生变化: 33, 差分像素1: 749, 差分像素2: 9549
图像已保存: D:\download\gene\ppt_real

帧发生变化: 27, 差分像素1: 10549, 差分像素2: 36642
图像已保存: D:\download\gene\ppt_real\output_directory\9_27.jpg
帧发生变化: 28, 差分像素1: 10549, 差分像素2: 35366
图像已保存: D:\download\gene\ppt_real\output_directory\9_28.jpg
帧发生变化: 29, 差分像素1: 0, 差分像素2: 14205
图像已保存: D:\download\gene\ppt_real\output_directory\9_29.jpg
帧发生变化: 30, 差分像素1: 0, 差分像素2: 10881
图像已保存: D:\download\gene\ppt_real\output_directory\9_30.jpg
帧发生变化: 31, 差分像素1: 0, 差分像素2: 8697
图像已保存: D:\download\gene\ppt_real\output_directory\9_31.jpg
帧发生变化: 32, 差分像素1: 2124, 差分像素2: 770
图像已保存: D:\download\gene\ppt_real\output_directory\9_32.jpg
帧发生变化: 33, 差分像素1: 2124, 差分像素2: 7167
图像已保存: D:\download\gene\ppt_real\output_directory\9_33.jpg
帧发生变化: 34, 差分像素1: 872, 差分像素2: 4
图像已保存: D:\download\gene\ppt_real\output_directory\9_34.jpg
帧发生变化: 35, 差分像素1: 1242, 差分像素2: 9841
图像已保存: D:\download\gene\ppt_real\output_directory\9_35.jpg
帧发生变化: 36, 差分像素1: 365, 差分像素2: 7684
图像已保存: D:\download\gene\ppt_real\output_directory\9_36.jpg
帧发生变化: 37, 差分像素1: 488, 差分像素2: 10319
图像已保存: D:\download\gene

帧发生变化: 19, 差分像素1: 1156, 差分像素2: 14869
图像已保存: D:\download\gene\ppt_real\output_directory\10_19.jpg
帧发生变化: 20, 差分像素1: 678, 差分像素2: 5952
图像已保存: D:\download\gene\ppt_real\output_directory\10_20.jpg
帧发生变化: 21, 差分像素1: 1635, 差分像素2: 15247
图像已保存: D:\download\gene\ppt_real\output_directory\10_21.jpg
帧发生变化: 22, 差分像素1: 1636, 差分像素2: 15244
图像已保存: D:\download\gene\ppt_real\output_directory\10_22.jpg
帧发生变化: 23, 差分像素1: 10454, 差分像素2: 37214
图像已保存: D:\download\gene\ppt_real\output_directory\10_23.jpg
帧发生变化: 24, 差分像素1: 10549, 差分像素2: 37606
图像已保存: D:\download\gene\ppt_real\output_directory\10_24.jpg
帧发生变化: 25, 差分像素1: 0, 差分像素2: 18589
图像已保存: D:\download\gene\ppt_real\output_directory\10_25.jpg
帧发生变化: 26, 差分像素1: 0, 差分像素2: 34206
图像已保存: D:\download\gene\ppt_real\output_directory\10_26.jpg
帧发生变化: 27, 差分像素1: 0, 差分像素2: 33050
图像已保存: D:\download\gene\ppt_real\output_directory\10_27.jpg
帧发生变化: 28, 差分像素1: 1702, 差分像素2: 12180
图像已保存: D:\download\gene\ppt_real\output_directory\10_28.jpg
帧发生变化: 29, 差分像素1: 1987, 差分像素2: 4828
图像已

帧发生变化: 16, 差分像素1: 0, 差分像素2: 10763
图像已保存: D:\download\gene\ppt_real\output_directory\12_16.jpg
帧发生变化: 17, 差分像素1: 0, 差分像素2: 11620
图像已保存: D:\download\gene\ppt_real\output_directory\12_17.jpg
帧发生变化: 18, 差分像素1: 1759, 差分像素2: 11195
图像已保存: D:\download\gene\ppt_real\output_directory\12_18.jpg
帧发生变化: 19, 差分像素1: 1753, 差分像素2: 18625
图像已保存: D:\download\gene\ppt_real\output_directory\12_19.jpg
帧发生变化: 20, 差分像素1: 2589, 差分像素2: 32879
图像已保存: D:\download\gene\ppt_real\output_directory\12_20.jpg
帧发生变化: 21, 差分像素1: 2605, 差分像素2: 30034
图像已保存: D:\download\gene\ppt_real\output_directory\12_21.jpg
帧发生变化: 22, 差分像素1: 1511, 差分像素2: 31378
图像已保存: D:\download\gene\ppt_real\output_directory\12_22.jpg
帧发生变化: 23, 差分像素1: 1441, 差分像素2: 29029
图像已保存: D:\download\gene\ppt_real\output_directory\12_23.jpg
帧发生变化: 24, 差分像素1: 0, 差分像素2: 8560
图像已保存: D:\download\gene\ppt_real\output_directory\12_24.jpg
帧发生变化: 25, 差分像素1: 0, 差分像素2: 9926
图像已保存: D:\download\gene\ppt_real\output_directory\12_25.jpg
帧发生变化: 26, 差分像素1: 10549, 差分像素2: 34875
图像已保存:

KeyboardInterrupt: 

In [6]:
import os
from datetime import datetime
import cv2
from fpdf import FPDF
import re

# 指定视频文件所在的目录路径 (绝对路径)
directory_path = r'D:\download\gene\ppt_real'

# 指定输出图像和PDF的目录 (绝对路径)
output_directory = os.path.join(directory_path, 'output_directory')
os.makedirs(output_directory, exist_ok=True)  # 如果目录不存在，则创建

# 获取目录下所有文件的列表
file_list = os.listdir(directory_path)

# 按照文件的修改时间进行排序
file_list.sort(key=lambda x: os.path.getmtime(os.path.join(directory_path, x)))

# 遍历排序后的文件列表
for file_name in file_list:
    file_path = os.path.join(directory_path, file_name)

    # 检查是否为文件而非子目录
    if not os.path.isfile(file_path):
        continue  # 跳过非文件项

    # 获取文件的修改时间
    modified_time = datetime.fromtimestamp(os.path.getmtime(file_path))
    print(f"正在处理文件: {file_path}")
    print(f"文件名：{file_name}，修改时间：{modified_time}")

    # 打开视频文件
    cap = cv2.VideoCapture(file_path)

    # 检查视频是否成功打开
    if not cap.isOpened():
        print(f"无法打开视频文件: {file_path}")
        continue  # 跳过无法打开的视频文件

    # 初始化变量
    prev_frame_gray_1 = None
    prev_frame_gray_2 = None
    page_images = []
    i = 0

    while True:
        ret, frame = cap.read()

        if not ret:
            break  # 读取完毕或发生错误

        # 检查帧是否为空
        if frame is None:
            continue

        # 确保感兴趣区域的坐标在帧的边界内
        if frame.shape[0] < 720 or frame.shape[1] < 1280:
            print(f"帧尺寸不足，跳过该帧，帧尺寸：{frame.shape}")
            continue

        # 调整感兴趣区域的坐标以适应帧的实际尺寸
        roi_1 = frame[480:660, 1060:1200]  # 感兴趣区域1，往上移动100像素
        roi_2 = frame[280:500, 340:540]    # 感兴趣区域2，扩大一些

        # 在帧上绘制感兴趣区域的矩形框
        cv2.rectangle(frame, (1060, 480), (1200, 660), (0, 255, 0), 2)  # 绘制感兴趣区域1
        cv2.rectangle(frame, (340, 280), (540, 500), (0, 255, 0), 2)    # 绘制感兴趣区域2

        # 将感兴趣区域转换为灰度图像
        roi_gray_1 = cv2.cvtColor(roi_1, cv2.COLOR_BGR2GRAY)
        roi_gray_2 = cv2.cvtColor(roi_2, cv2.COLOR_BGR2GRAY)

        # 初始化前一帧灰度图像
        if prev_frame_gray_1 is None:
            prev_frame_gray_1 = roi_gray_1.copy()
            prev_frame_gray_2 = roi_gray_2.copy()
            continue  # 跳过第一帧的差分检测

        # 计算当前帧和前一帧的差分图像
        frame_diff_1 = cv2.absdiff(roi_gray_1, prev_frame_gray_1)
        frame_diff_2 = cv2.absdiff(roi_gray_2, prev_frame_gray_2)

        # 对差分图像进行阈值处理
        _, threshold_1 = cv2.threshold(frame_diff_1, 30, 255, cv2.THRESH_BINARY)
        _, threshold_2 = cv2.threshold(frame_diff_2, 30, 255, cv2.THRESH_BINARY)

        # 统计阈值图像中非零像素的数量
        diff_pixels_1 = cv2.countNonZero(threshold_1)
        diff_pixels_2 = cv2.countNonZero(threshold_2)

        # 如果两个区域的差分像素数量都超过阈值，则认为帧发生了变化
        if (diff_pixels_1 > 100 and diff_pixels_2 > 300) or diff_pixels_1 > 150 or diff_pixels_2 > 1000:
            print(f"帧发生变化: {i + 1}, 差分像素1: {diff_pixels_1}, 差分像素2: {diff_pixels_2}")

            # 提取文件名中的编号部分
            match = re.search(r'第(\d+)讲', file_name)
            if match:
                lecture_number = match.group(1)
            else:
                lecture_number = 'unknown'

            # 生成简化的文件名
            simplified_file_name = f"{lecture_number}_{i + 1}.jpg"
            page_path = os.path.join(output_directory, simplified_file_name)

            # 尝试保存图像
            try:
                # 检查输出目录是否存在
                if not os.path.exists(output_directory):
                    os.makedirs(output_directory, exist_ok=True)
                    print(f"创建输出目录: {output_directory}")

                # 保存图像
                success = cv2.imwrite(page_path, frame)
                if success:
                    print(f"图像已保存: {page_path}")
                    page_images.append(page_path)
                else:
                    print(f"图像保存失败: {page_path}")
            except Exception as e:
                print(f"保存图像时发生错误: {str(e)}")
            i += 1

        # 更新前一帧
        prev_frame_gray_1 = roi_gray_1.copy()
        prev_frame_gray_2 = roi_gray_2.copy()

        # 显示当前帧 (可选，运行时可关闭窗口加快速度)
        cv2.imshow("Frame", frame)

        # 跳过一定数量的帧以加快处理速度
        current_frame = cap.get(cv2.CAP_PROP_POS_FRAMES)
        max_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        # 计算目标帧的索引（例如跳500帧）
        target_frame = current_frame + 500
        if target_frame >= max_frames:
            break  # 超过总帧数，结束处理
        cap.set(cv2.CAP_PROP_POS_FRAMES, target_frame)

        # 按下 'q' 键退出循环 (可选，已注释)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    # 创建PDF文件
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)

    for image_path in page_images:
        if os.path.exists(image_path):
            pdf.add_page()
            # 计算图片的宽度和高度，以适应PDF页面
            pdf_width = pdf.w - 20  # 左右各10的边距
            pdf.image(image_path, x=10, y=10, w=pdf_width)
        else:
            print(f"图像文件不存在: {image_path}")

    # 保存PDF
    pdf_file_name = os.path.splitext(file_name)[0] + '.pdf'
    pdf_path = os.path.join(output_directory, pdf_file_name)
    try:
        pdf.output(pdf_path)
        print(f"已保存PDF: {pdf_path}")
    except Exception as e:
        print(f"保存PDF时发生错误: {str(e)}")

正在处理文件: D:\download\gene\ppt_real\动植物基因表达调控(研)_于明_动植物基因表达调控(研)(第2讲)_1.mp4
文件名：动植物基因表达调控(研)_于明_动植物基因表达调控(研)(第2讲)_1.mp4，修改时间：2025-01-06 21:10:33.576509
帧发生变化: 1, 差分像素1: 0, 差分像素2: 3837
图像已保存: D:\download\gene\ppt_real\output_directory\2_1.jpg
帧发生变化: 2, 差分像素1: 0, 差分像素2: 2869
图像已保存: D:\download\gene\ppt_real\output_directory\2_2.jpg
帧发生变化: 3, 差分像素1: 0, 差分像素2: 9391
图像已保存: D:\download\gene\ppt_real\output_directory\2_3.jpg
帧发生变化: 4, 差分像素1: 731, 差分像素2: 13745
图像已保存: D:\download\gene\ppt_real\output_directory\2_4.jpg
帧发生变化: 5, 差分像素1: 732, 差分像素2: 8492
图像已保存: D:\download\gene\ppt_real\output_directory\2_5.jpg
帧发生变化: 6, 差分像素1: 2235, 差分像素2: 4635
图像已保存: D:\download\gene\ppt_real\output_directory\2_6.jpg
帧发生变化: 7, 差分像素1: 4174, 差分像素2: 12816
图像已保存: D:\download\gene\ppt_real\output_directory\2_7.jpg
帧发生变化: 8, 差分像素1: 2022, 差分像素2: 16376
图像已保存: D:\download\gene\ppt_real\output_directory\2_8.jpg
帧发生变化: 9, 差分像素1: 68, 差分像素2: 14128
图像已保存: D:\download\gene\ppt_real\output_directory\2_9.jpg
帧发生变化: 10, 差分像素1: 2423

帧发生变化: 32, 差分像素1: 669, 差分像素2: 34681
图像已保存: D:\download\gene\ppt_real\output_directory\1_32.jpg
帧发生变化: 33, 差分像素1: 761, 差分像素2: 12770
图像已保存: D:\download\gene\ppt_real\output_directory\1_33.jpg
帧发生变化: 34, 差分像素1: 761, 差分像素2: 7323
图像已保存: D:\download\gene\ppt_real\output_directory\1_34.jpg
帧发生变化: 35, 差分像素1: 0, 差分像素2: 5218
图像已保存: D:\download\gene\ppt_real\output_directory\1_35.jpg
帧发生变化: 36, 差分像素1: 2481, 差分像素2: 3
图像已保存: D:\download\gene\ppt_real\output_directory\1_36.jpg
帧发生变化: 37, 差分像素1: 5034, 差分像素2: 24291
图像已保存: D:\download\gene\ppt_real\output_directory\1_37.jpg
帧发生变化: 38, 差分像素1: 3719, 差分像素2: 21924
图像已保存: D:\download\gene\ppt_real\output_directory\1_38.jpg
帧发生变化: 39, 差分像素1: 5447, 差分像素2: 20868
图像已保存: D:\download\gene\ppt_real\output_directory\1_39.jpg
帧发生变化: 40, 差分像素1: 4983, 差分像素2: 33753
图像已保存: D:\download\gene\ppt_real\output_directory\1_40.jpg
帧发生变化: 41, 差分像素1: 998, 差分像素2: 31983
图像已保存: D:\download\gene\ppt_real\output_directory\1_41.jpg
帧发生变化: 42, 差分像素1: 386, 差分像素2: 16814
图像已保存: D:\downloa

帧发生变化: 18, 差分像素1: 3613, 差分像素2: 4682
图像已保存: D:\download\gene\ppt_real\output_directory\5_18.jpg
帧发生变化: 19, 差分像素1: 643, 差分像素2: 0
图像已保存: D:\download\gene\ppt_real\output_directory\5_19.jpg
帧发生变化: 20, 差分像素1: 643, 差分像素2: 4676
图像已保存: D:\download\gene\ppt_real\output_directory\5_20.jpg
帧发生变化: 21, 差分像素1: 0, 差分像素2: 3924
图像已保存: D:\download\gene\ppt_real\output_directory\5_21.jpg
帧发生变化: 22, 差分像素1: 700, 差分像素2: 1368
图像已保存: D:\download\gene\ppt_real\output_directory\5_22.jpg
帧发生变化: 23, 差分像素1: 2022, 差分像素2: 3596
图像已保存: D:\download\gene\ppt_real\output_directory\5_23.jpg
帧发生变化: 24, 差分像素1: 1372, 差分像素2: 4569
图像已保存: D:\download\gene\ppt_real\output_directory\5_24.jpg
帧发生变化: 25, 差分像素1: 1564, 差分像素2: 15161
图像已保存: D:\download\gene\ppt_real\output_directory\5_25.jpg
帧发生变化: 26, 差分像素1: 1564, 差分像素2: 15166
图像已保存: D:\download\gene\ppt_real\output_directory\5_26.jpg
帧发生变化: 27, 差分像素1: 1558, 差分像素2: 15163
图像已保存: D:\download\gene\ppt_real\output_directory\5_27.jpg
帧发生变化: 28, 差分像素1: 4106, 差分像素2: 42400
图像已保存: D:\download\

帧发生变化: 19, 差分像素1: 849, 差分像素2: 14408
图像已保存: D:\download\gene\ppt_real\output_directory\6_19.jpg
帧发生变化: 20, 差分像素1: 4989, 差分像素2: 10533
图像已保存: D:\download\gene\ppt_real\output_directory\6_20.jpg
帧发生变化: 21, 差分像素1: 154, 差分像素2: 20
图像已保存: D:\download\gene\ppt_real\output_directory\6_21.jpg
帧发生变化: 22, 差分像素1: 166, 差分像素2: 24
图像已保存: D:\download\gene\ppt_real\output_directory\6_22.jpg
帧发生变化: 23, 差分像素1: 6893, 差分像素2: 12747
图像已保存: D:\download\gene\ppt_real\output_directory\6_23.jpg
帧发生变化: 24, 差分像素1: 4189, 差分像素2: 38968
图像已保存: D:\download\gene\ppt_real\output_directory\6_24.jpg
帧发生变化: 25, 差分像素1: 0, 差分像素2: 31250
图像已保存: D:\download\gene\ppt_real\output_directory\6_25.jpg
帧发生变化: 26, 差分像素1: 1293, 差分像素2: 33926
图像已保存: D:\download\gene\ppt_real\output_directory\6_26.jpg
帧发生变化: 27, 差分像素1: 0, 差分像素2: 7298
图像已保存: D:\download\gene\ppt_real\output_directory\6_27.jpg
帧发生变化: 28, 差分像素1: 635, 差分像素2: 4316
图像已保存: D:\download\gene\ppt_real\output_directory\6_28.jpg
帧发生变化: 29, 差分像素1: 752, 差分像素2: 1451
图像已保存: D:\download\gene

帧发生变化: 17, 差分像素1: 2805, 差分像素2: 23667
图像已保存: D:\download\gene\ppt_real\output_directory\9_17.jpg
帧发生变化: 18, 差分像素1: 234, 差分像素2: 20717
图像已保存: D:\download\gene\ppt_real\output_directory\9_18.jpg
帧发生变化: 19, 差分像素1: 839, 差分像素2: 0
图像已保存: D:\download\gene\ppt_real\output_directory\9_19.jpg
帧发生变化: 20, 差分像素1: 4453, 差分像素2: 18654
图像已保存: D:\download\gene\ppt_real\output_directory\9_20.jpg
帧发生变化: 21, 差分像素1: 4620, 差分像素2: 16109
图像已保存: D:\download\gene\ppt_real\output_directory\9_21.jpg
帧发生变化: 22, 差分像素1: 2556, 差分像素2: 4933
图像已保存: D:\download\gene\ppt_real\output_directory\9_22.jpg
帧发生变化: 23, 差分像素1: 1701, 差分像素2: 8748
图像已保存: D:\download\gene\ppt_real\output_directory\9_23.jpg
帧发生变化: 24, 差分像素1: 0, 差分像素2: 10046
图像已保存: D:\download\gene\ppt_real\output_directory\9_24.jpg
帧发生变化: 25, 差分像素1: 3936, 差分像素2: 10042
图像已保存: D:\download\gene\ppt_real\output_directory\9_25.jpg
帧发生变化: 26, 差分像素1: 3547, 差分像素2: 8884
图像已保存: D:\download\gene\ppt_real\output_directory\9_26.jpg
帧发生变化: 27, 差分像素1: 0, 差分像素2: 5015
图像已保存: D:\download\

...

帧发生变化: 6, 差分像素1: 398, 差分像素2: 3788
图像已保存: D:\download\gene\ppt_real\output_directory\19_6.jpg
帧发生变化: 7, 差分像素1: 405, 差分像素2: 3898
图像已保存: D:\download\gene\ppt_real\output_directory\19_7.jpg
帧发生变化: 8, 差分像素1: 753, 差分像素2: 3754
图像已保存: D:\download\gene\ppt_real\output_directory\19_8.jpg
帧发生变化: 9, 差分像素1: 1010, 差分像素2: 4071
图像已保存: D:\download\gene\ppt_real\output_directory\19_9.jpg
帧发生变化: 10, 差分像素1: 0, 差分像素2: 3751
图像已保存: D:\download\gene\ppt_real\output_directory\19_10.jpg
帧发生变化: 11, 差分像素1: 0, 差分像素2: 3937
图像已保存: D:\download\gene\ppt_real\output_directory\19_11.jpg
帧发生变化: 12, 差分像素1: 17059, 差分像素2: 16005
图像已保存: D:\download\gene\ppt_real\output_directory\19_12.jpg
帧发生变化: 13, 差分像素1: 7191, 差分像素2: 14090
图像已保存: D:\download\gene\ppt_real\output_directory\19_13.jpg
帧发生变化: 14, 差分像素1: 2692, 差分像素2: 11834
图像已保存: D:\download\gene\ppt_real\output_directory\19_14.jpg
帧发生变化: 15, 差分像素1: 5185, 差分像素2: 8542
图像已保存: D:\download\gene\ppt_real\output_directory\19_15.jpg
帧发生变化: 16, 差分像素1: 7241, 差分像素2: 9931
图像已保存: D:\downloa

帧发生变化: 12, 差分像素1: 0, 差分像素2: 2175
图像已保存: D:\download\gene\ppt_real\output_directory\23_12.jpg
帧发生变化: 13, 差分像素1: 630, 差分像素2: 2576
图像已保存: D:\download\gene\ppt_real\output_directory\23_13.jpg
帧发生变化: 14, 差分像素1: 0, 差分像素2: 14451
图像已保存: D:\download\gene\ppt_real\output_directory\23_14.jpg
帧发生变化: 15, 差分像素1: 15738, 差分像素2: 20200
图像已保存: D:\download\gene\ppt_real\output_directory\23_15.jpg
帧发生变化: 16, 差分像素1: 0, 差分像素2: 40251
图像已保存: D:\download\gene\ppt_real\output_directory\23_16.jpg
帧发生变化: 17, 差分像素1: 4014, 差分像素2: 34197
图像已保存: D:\download\gene\ppt_real\output_directory\23_17.jpg
帧发生变化: 18, 差分像素1: 5410, 差分像素2: 37975
图像已保存: D:\download\gene\ppt_real\output_directory\23_18.jpg
帧发生变化: 19, 差分像素1: 3649, 差分像素2: 19272
图像已保存: D:\download\gene\ppt_real\output_directory\23_19.jpg
帧发生变化: 20, 差分像素1: 2660, 差分像素2: 14560
图像已保存: D:\download\gene\ppt_real\output_directory\23_20.jpg
帧发生变化: 21, 差分像素1: 3326, 差分像素2: 14300
图像已保存: D:\download\gene\ppt_real\output_directory\23_21.jpg
帧发生变化: 22, 差分像素1: 4542, 差分像素2: 10460
图像已保

帧发生变化: 41, 差分像素1: 14291, 差分像素2: 7843
图像已保存: D:\download\gene\ppt_real\output_directory\24_41.jpg
已保存PDF: D:\download\gene\ppt_real\output_directory\动植物基因表达调控(研)_于明_动植物基因表达调控(研)(第24讲)_1.pdf
正在处理文件: D:\download\gene\ppt_real\动植物基因表达调控(研)_于明_动植物基因表达调控(研)(第25讲)_1.mp4
文件名：动植物基因表达调控(研)_于明_动植物基因表达调控(研)(第25讲)_1.mp4，修改时间：2025-01-06 22:09:53.454425
帧发生变化: 1, 差分像素1: 423, 差分像素2: 8638
图像已保存: D:\download\gene\ppt_real\output_directory\25_1.jpg
帧发生变化: 2, 差分像素1: 581, 差分像素2: 9049
图像已保存: D:\download\gene\ppt_real\output_directory\25_2.jpg
帧发生变化: 3, 差分像素1: 0, 差分像素2: 8923
图像已保存: D:\download\gene\ppt_real\output_directory\25_3.jpg
帧发生变化: 4, 差分像素1: 648, 差分像素2: 6844
图像已保存: D:\download\gene\ppt_real\output_directory\25_4.jpg
帧发生变化: 5, 差分像素1: 0, 差分像素2: 7079
图像已保存: D:\download\gene\ppt_real\output_directory\25_5.jpg
帧发生变化: 6, 差分像素1: 443, 差分像素2: 7628
图像已保存: D:\download\gene\ppt_real\output_directory\25_6.jpg
帧发生变化: 7, 差分像素1: 681, 差分像素2: 8246
图像已保存: D:\download\gene\ppt_real\output_directory\25_7.jpg
帧发生变化: 8, 差分

帧发生变化: 40, 差分像素1: 872, 差分像素2: 4095
图像已保存: D:\download\gene\ppt_real\output_directory\26_40.jpg
帧发生变化: 41, 差分像素1: 49, 差分像素2: 5064
图像已保存: D:\download\gene\ppt_real\output_directory\26_41.jpg
帧发生变化: 42, 差分像素1: 12926, 差分像素2: 40327
图像已保存: D:\download\gene\ppt_real\output_directory\26_42.jpg
帧发生变化: 43, 差分像素1: 0, 差分像素2: 2454
图像已保存: D:\download\gene\ppt_real\output_directory\26_43.jpg
帧发生变化: 44, 差分像素1: 13103, 差分像素2: 40043
图像已保存: D:\download\gene\ppt_real\output_directory\26_44.jpg
帧发生变化: 45, 差分像素1: 0, 差分像素2: 6772
图像已保存: D:\download\gene\ppt_real\output_directory\26_45.jpg
帧发生变化: 46, 差分像素1: 23837, 差分像素2: 37236
图像已保存: D:\download\gene\ppt_real\output_directory\26_46.jpg
已保存PDF: D:\download\gene\ppt_real\output_directory\动植物基因表达调控(研)_于明_动植物基因表达调控(研)(第26讲)_1.pdf
正在处理文件: D:\download\gene\ppt_real\动植物基因表达调控(研)_于明_动植物基因表达调控(研)(第27讲)_1.mp4
文件名：动植物基因表达调控(研)_于明_动植物基因表达调控(研)(第27讲)_1.mp4，修改时间：2025-01-06 22:14:31.277979
帧发生变化: 1, 差分像素1: 0, 差分像素2: 3612
图像已保存: D:\download\gene\ppt_real\output_directory\27_1.

帧发生变化: 8, 差分像素1: 3818, 差分像素2: 14207
图像已保存: D:\download\gene\ppt_real\output_directory\28_8.jpg
帧发生变化: 9, 差分像素1: 4535, 差分像素2: 30426
图像已保存: D:\download\gene\ppt_real\output_directory\28_9.jpg
帧发生变化: 10, 差分像素1: 6355, 差分像素2: 29644
图像已保存: D:\download\gene\ppt_real\output_directory\28_10.jpg
帧发生变化: 11, 差分像素1: 4900, 差分像素2: 13904
图像已保存: D:\download\gene\ppt_real\output_directory\28_11.jpg
帧发生变化: 12, 差分像素1: 4465, 差分像素2: 24258
图像已保存: D:\download\gene\ppt_real\output_directory\28_12.jpg
帧发生变化: 13, 差分像素1: 4659, 差分像素2: 24642
图像已保存: D:\download\gene\ppt_real\output_directory\28_13.jpg
帧发生变化: 14, 差分像素1: 1881, 差分像素2: 15584
图像已保存: D:\download\gene\ppt_real\output_directory\28_14.jpg
帧发生变化: 15, 差分像素1: 618, 差分像素2: 0
图像已保存: D:\download\gene\ppt_real\output_directory\28_15.jpg
帧发生变化: 16, 差分像素1: 1159, 差分像素2: 25349
图像已保存: D:\download\gene\ppt_real\output_directory\28_16.jpg
帧发生变化: 17, 差分像素1: 617, 差分像素2: 27268
图像已保存: D:\download\gene\ppt_real\output_directory\28_17.jpg
帧发生变化: 18, 差分像素1: 0, 差分像素2: 24222
图像已保存:

In [7]:
import os
from datetime import datetime
import cv2
from fpdf import FPDF
import re

# 指定视频文件所在的目录路径 (绝对路径)
directory_path = r'D:\download\gene\ppt_real'

# 指定输出图像和PDF的目录 (绝对路径)
output_directory = os.path.join(directory_path, 'output_directory')
os.makedirs(output_directory, exist_ok=True)  # 如果目录不存在，则创建

# 获取目录下所有文件的列表
file_list = os.listdir(directory_path)

# 按照文件的修改时间进行排序
file_list.sort(key=lambda x: os.path.getmtime(os.path.join(directory_path, x)))

# 遍历排序后的文件列表
for file_name in file_list:
    file_path = os.path.join(directory_path, file_name)

    # 检查是否为文件而非子目录
    if not os.path.isfile(file_path):
        continue  # 跳过非文件项

    # 获取文件的修改时间
    modified_time = datetime.fromtimestamp(os.path.getmtime(file_path))
    print(f"正在处理文件: {file_path}")
    print(f"文件名：{file_name}，修改时间：{modified_time}")

    # 打开视频文件
    cap = cv2.VideoCapture(file_path)

    # 检查视频是否成功打开
    if not cap.isOpened():
        print(f"无法打开视频文件: {file_path}")
        continue  # 跳过无法打开的视频文件

    # 初始化变量
    prev_frame_gray_1 = None
    prev_frame_gray_2 = None
    page_images = []
    i = 0

    while True:
        ret, frame = cap.read()

        if not ret:
            break  # 读取完毕或发生错误

        # 检查帧是否为空
        if frame is None:
            continue

        # 确保感兴趣区域的坐标在帧的边界内
        if frame.shape[0] < 720 or frame.shape[1] < 1280:
            print(f"帧尺寸不足，跳过该帧，帧尺寸：{frame.shape}")
            continue

        # 调整感兴趣区域的坐标以适应帧的实际尺寸
        roi_1 = frame[480:660, 1060:1200]  # 感兴趣区域1，往上移动100像素
        roi_2 = frame[280:500, 340:540]    # 感兴趣区域2，扩大一些

        # 将感兴趣区域转换为灰度图像
        roi_gray_1 = cv2.cvtColor(roi_1, cv2.COLOR_BGR2GRAY)
        roi_gray_2 = cv2.cvtColor(roi_2, cv2.COLOR_BGR2GRAY)

        # 初始化前一帧灰度图像
        if prev_frame_gray_1 is None:
            prev_frame_gray_1 = roi_gray_1.copy()
            prev_frame_gray_2 = roi_gray_2.copy()
            continue  # 跳过第一帧的差分检测

        # 计算当前帧和前一帧的差分图像
        frame_diff_1 = cv2.absdiff(roi_gray_1, prev_frame_gray_1)
        frame_diff_2 = cv2.absdiff(roi_gray_2, prev_frame_gray_2)

        # 对差分图像进行阈值处理
        _, threshold_1 = cv2.threshold(frame_diff_1, 30, 255, cv2.THRESH_BINARY)
        _, threshold_2 = cv2.threshold(frame_diff_2, 30, 255, cv2.THRESH_BINARY)

        # 统计阈值图像中非零像素的数量
        diff_pixels_1 = cv2.countNonZero(threshold_1)
        diff_pixels_2 = cv2.countNonZero(threshold_2)

        # 如果两个区域的差分像素数量都超过阈值，则认为帧发生了变化
        if (diff_pixels_1 > 100 and diff_pixels_2 > 300) or diff_pixels_1 > 150 or diff_pixels_2 > 1000:
            print(f"帧发生变化: {i + 1}, 差分像素1: {diff_pixels_1}, 差分像素2: {diff_pixels_2}")

            # 提取文件名中的编号部分
            match = re.search(r'第(\d+)讲', file_name)
            if match:
                lecture_number = match.group(1)
            else:
                lecture_number = 'unknown'

            # 生成简化的文件名
            simplified_file_name = f"{lecture_number}_{i + 1}.jpg"
            page_path = os.path.join(output_directory, simplified_file_name)

            # 尝试保存图像
            try:
                # 检查输出目录是否存在
                if not os.path.exists(output_directory):
                    os.makedirs(output_directory, exist_ok=True)
                    print(f"创建输出目录: {output_directory}")

                # 保存图像
                success = cv2.imwrite(page_path, frame)
                if success:
                    print(f"图像已保存: {page_path}")
                    page_images.append(page_path)
                else:
                    print(f"图像保存失败: {page_path}")
            except Exception as e:
                print(f"保存图像时发生错误: {str(e)}")
            i += 1

        # 更新前一帧
        prev_frame_gray_1 = roi_gray_1.copy()
        prev_frame_gray_2 = roi_gray_2.copy()

        # 显示当前帧 (可选，运行时可关闭窗口加快速度)
        # cv2.imshow("Frame", frame)

        # 跳过一定数量的帧以加快处理速度
        current_frame = cap.get(cv2.CAP_PROP_POS_FRAMES)
        max_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        # 计算目标帧的索引（例如跳500帧）
        target_frame = current_frame + 500
        if target_frame >= max_frames:
            break  # 超过总帧数，结束处理
        cap.set(cv2.CAP_PROP_POS_FRAMES, target_frame)

        # 按下 'q' 键退出循环 (可选，已注释)
        # if cv2.waitKey(1) & 0xFF == ord('q'):
        #     break

    cap.release()
    cv2.destroyAllWindows()

    # 创建PDF文件
    pdf = FPDF(orientation='L', unit='mm', format='A4')  # 使用横向A4纸
    pdf.set_auto_page_break(auto=True, margin=15)

    for image_path in page_images:
        if os.path.exists(image_path):
            pdf.add_page()
            # 计算图片的宽度和高度，以适应PDF页面
            pdf_width = pdf.w - 20  # 左右各10的边距
            pdf_height = pdf.h - 20  # 上下各10的边距
            pdf.image(image_path, x=10, y=10, w=pdf_width, h=pdf_height)
        else:
            print(f"图像文件不存在: {image_path}")

    # 保存PDF
    pdf_file_name = os.path.splitext(file_name)[0] + '.pdf'
    pdf_path = os.path.join(output_directory, pdf_file_name)
    try:
        pdf.output(pdf_path)
        print(f"已保存PDF: {pdf_path}")
    except Exception as e:
        print(f"保存PDF时发生错误: {str(e)}")

正在处理文件: D:\download\gene\ppt_real\动植物基因表达调控(研)_于明_动植物基因表达调控(研)(第2讲)_1.mp4
文件名：动植物基因表达调控(研)_于明_动植物基因表达调控(研)(第2讲)_1.mp4，修改时间：2025-01-06 21:10:33.576509
帧发生变化: 1, 差分像素1: 0, 差分像素2: 4058
图像已保存: D:\download\gene\ppt_real\output_directory\2_1.jpg
帧发生变化: 2, 差分像素1: 0, 差分像素2: 2913
图像已保存: D:\download\gene\ppt_real\output_directory\2_2.jpg
帧发生变化: 3, 差分像素1: 0, 差分像素2: 9490
图像已保存: D:\download\gene\ppt_real\output_directory\2_3.jpg
帧发生变化: 4, 差分像素1: 731, 差分像素2: 13992
图像已保存: D:\download\gene\ppt_real\output_directory\2_4.jpg
帧发生变化: 5, 差分像素1: 732, 差分像素2: 8692
图像已保存: D:\download\gene\ppt_real\output_directory\2_5.jpg
帧发生变化: 6, 差分像素1: 2259, 差分像素2: 4816
图像已保存: D:\download\gene\ppt_real\output_directory\2_6.jpg
帧发生变化: 7, 差分像素1: 4407, 差分像素2: 13295
图像已保存: D:\download\gene\ppt_real\output_directory\2_7.jpg
帧发生变化: 8, 差分像素1: 2257, 差分像素2: 16835
图像已保存: D:\download\gene\ppt_real\output_directory\2_8.jpg
帧发生变化: 9, 差分像素1: 113, 差分像素2: 14277
图像已保存: D:\download\gene\ppt_real\output_directory\2_9.jpg
帧发生变化: 10, 差分像素1: 244

帧发生变化: 32, 差分像素1: 683, 差分像素2: 35806
图像已保存: D:\download\gene\ppt_real\output_directory\1_32.jpg
帧发生变化: 33, 差分像素1: 788, 差分像素2: 13145
图像已保存: D:\download\gene\ppt_real\output_directory\1_33.jpg
帧发生变化: 34, 差分像素1: 788, 差分像素2: 7613
图像已保存: D:\download\gene\ppt_real\output_directory\1_34.jpg
帧发生变化: 35, 差分像素1: 0, 差分像素2: 5301
图像已保存: D:\download\gene\ppt_real\output_directory\1_35.jpg
帧发生变化: 36, 差分像素1: 2514, 差分像素2: 13
图像已保存: D:\download\gene\ppt_real\output_directory\1_36.jpg
帧发生变化: 37, 差分像素1: 5125, 差分像素2: 24955
图像已保存: D:\download\gene\ppt_real\output_directory\1_37.jpg
帧发生变化: 38, 差分像素1: 3883, 差分像素2: 22439
图像已保存: D:\download\gene\ppt_real\output_directory\1_38.jpg
帧发生变化: 39, 差分像素1: 5617, 差分像素2: 21402
图像已保存: D:\download\gene\ppt_real\output_directory\1_39.jpg
帧发生变化: 40, 差分像素1: 5078, 差分像素2: 34511
图像已保存: D:\download\gene\ppt_real\output_directory\1_40.jpg
帧发生变化: 41, 差分像素1: 1141, 差分像素2: 32718
图像已保存: D:\download\gene\ppt_real\output_directory\1_41.jpg
帧发生变化: 42, 差分像素1: 512, 差分像素2: 17465
图像已保存: D:\downl

帧发生变化: 15, 差分像素1: 2302, 差分像素2: 6870
图像已保存: D:\download\gene\ppt_real\output_directory\5_15.jpg
帧发生变化: 16, 差分像素1: 3442, 差分像素2: 1018
图像已保存: D:\download\gene\ppt_real\output_directory\5_16.jpg
帧发生变化: 17, 差分像素1: 6596, 差分像素2: 5618
图像已保存: D:\download\gene\ppt_real\output_directory\5_17.jpg
帧发生变化: 18, 差分像素1: 3706, 差分像素2: 5060
图像已保存: D:\download\gene\ppt_real\output_directory\5_18.jpg
帧发生变化: 19, 差分像素1: 669, 差分像素2: 0
图像已保存: D:\download\gene\ppt_real\output_directory\5_19.jpg
帧发生变化: 20, 差分像素1: 669, 差分像素2: 5044
图像已保存: D:\download\gene\ppt_real\output_directory\5_20.jpg
帧发生变化: 21, 差分像素1: 0, 差分像素2: 4336
图像已保存: D:\download\gene\ppt_real\output_directory\5_21.jpg
帧发生变化: 22, 差分像素1: 735, 差分像素2: 1421
图像已保存: D:\download\gene\ppt_real\output_directory\5_22.jpg
帧发生变化: 23, 差分像素1: 2157, 差分像素2: 3820
图像已保存: D:\download\gene\ppt_real\output_directory\5_23.jpg
帧发生变化: 24, 差分像素1: 1500, 差分像素2: 4757
图像已保存: D:\download\gene\ppt_real\output_directory\5_24.jpg
帧发生变化: 25, 差分像素1: 1598, 差分像素2: 15780
图像已保存: D:\download\gen

帧发生变化: 16, 差分像素1: 16478, 差分像素2: 43233
图像已保存: D:\download\gene\ppt_real\output_directory\6_16.jpg
帧发生变化: 17, 差分像素1: 16479, 差分像素2: 43148
图像已保存: D:\download\gene\ppt_real\output_directory\6_17.jpg
帧发生变化: 18, 差分像素1: 16856, 差分像素2: 41194
图像已保存: D:\download\gene\ppt_real\output_directory\6_18.jpg
帧发生变化: 19, 差分像素1: 941, 差分像素2: 14793
图像已保存: D:\download\gene\ppt_real\output_directory\6_19.jpg
帧发生变化: 20, 差分像素1: 5222, 差分像素2: 10822
图像已保存: D:\download\gene\ppt_real\output_directory\6_20.jpg
帧发生变化: 21, 差分像素1: 156, 差分像素2: 21
图像已保存: D:\download\gene\ppt_real\output_directory\6_21.jpg
帧发生变化: 22, 差分像素1: 168, 差分像素2: 24
图像已保存: D:\download\gene\ppt_real\output_directory\6_22.jpg
帧发生变化: 23, 差分像素1: 7184, 差分像素2: 13319
图像已保存: D:\download\gene\ppt_real\output_directory\6_23.jpg
帧发生变化: 24, 差分像素1: 4353, 差分像素2: 40129
图像已保存: D:\download\gene\ppt_real\output_directory\6_24.jpg
帧发生变化: 25, 差分像素1: 0, 差分像素2: 32127
图像已保存: D:\download\gene\ppt_real\output_directory\6_25.jpg
帧发生变化: 26, 差分像素1: 1386, 差分像素2: 34995
图像已保存: D:\do

帧发生变化: 14, 差分像素1: 2275, 差分像素2: 9639
图像已保存: D:\download\gene\ppt_real\output_directory\9_14.jpg
帧发生变化: 15, 差分像素1: 3254, 差分像素2: 0
图像已保存: D:\download\gene\ppt_real\output_directory\9_15.jpg
帧发生变化: 16, 差分像素1: 6045, 差分像素2: 18188
图像已保存: D:\download\gene\ppt_real\output_directory\9_16.jpg
帧发生变化: 17, 差分像素1: 4861, 差分像素2: 18884
图像已保存: D:\download\gene\ppt_real\output_directory\9_17.jpg
帧发生变化: 18, 差分像素1: 2845, 差分像素2: 24319
图像已保存: D:\download\gene\ppt_real\output_directory\9_18.jpg
帧发生变化: 19, 差分像素1: 242, 差分像素2: 21382
图像已保存: D:\download\gene\ppt_real\output_directory\9_19.jpg
帧发生变化: 20, 差分像素1: 936, 差分像素2: 0
图像已保存: D:\download\gene\ppt_real\output_directory\9_20.jpg
帧发生变化: 21, 差分像素1: 4630, 差分像素2: 19246
图像已保存: D:\download\gene\ppt_real\output_directory\9_21.jpg
帧发生变化: 22, 差分像素1: 4732, 差分像素2: 16334
图像已保存: D:\download\gene\ppt_real\output_directory\9_22.jpg
帧发生变化: 23, 差分像素1: 2746, 差分像素2: 5183
图像已保存: D:\download\gene\ppt_real\output_directory\9_23.jpg
帧发生变化: 24, 差分像素1: 1883, 差分像素2: 8996
图像已保存: D:\downlo

...

帧发生变化: 3, 差分像素1: 0, 差分像素2: 7479
图像已保存: D:\download\gene\ppt_real\output_directory\19_3.jpg
帧发生变化: 4, 差分像素1: 0, 差分像素2: 7306
图像已保存: D:\download\gene\ppt_real\output_directory\19_4.jpg
帧发生变化: 5, 差分像素1: 409, 差分像素2: 3654
图像已保存: D:\download\gene\ppt_real\output_directory\19_5.jpg
帧发生变化: 6, 差分像素1: 398, 差分像素2: 3865
图像已保存: D:\download\gene\ppt_real\output_directory\19_6.jpg
帧发生变化: 7, 差分像素1: 416, 差分像素2: 3958
图像已保存: D:\download\gene\ppt_real\output_directory\19_7.jpg
帧发生变化: 8, 差分像素1: 754, 差分像素2: 3844
图像已保存: D:\download\gene\ppt_real\output_directory\19_8.jpg
帧发生变化: 9, 差分像素1: 1010, 差分像素2: 4163
图像已保存: D:\download\gene\ppt_real\output_directory\19_9.jpg
帧发生变化: 10, 差分像素1: 0, 差分像素2: 3800
图像已保存: D:\download\gene\ppt_real\output_directory\19_10.jpg
帧发生变化: 11, 差分像素1: 0, 差分像素2: 4028
图像已保存: D:\download\gene\ppt_real\output_directory\19_11.jpg
帧发生变化: 12, 差分像素1: 17515, 差分像素2: 16420
图像已保存: D:\download\gene\ppt_real\output_directory\19_12.jpg
帧发生变化: 13, 差分像素1: 7426, 差分像素2: 14603
图像已保存: D:\download\gene\ppt_rea

帧发生变化: 9, 差分像素1: 731, 差分像素2: 9122
图像已保存: D:\download\gene\ppt_real\output_directory\23_9.jpg
帧发生变化: 10, 差分像素1: 169, 差分像素2: 2275
图像已保存: D:\download\gene\ppt_real\output_directory\23_10.jpg
帧发生变化: 11, 差分像素1: 1172, 差分像素2: 2737
图像已保存: D:\download\gene\ppt_real\output_directory\23_11.jpg
帧发生变化: 12, 差分像素1: 0, 差分像素2: 2239
图像已保存: D:\download\gene\ppt_real\output_directory\23_12.jpg
帧发生变化: 13, 差分像素1: 630, 差分像素2: 2655
图像已保存: D:\download\gene\ppt_real\output_directory\23_13.jpg
帧发生变化: 14, 差分像素1: 0, 差分像素2: 14772
图像已保存: D:\download\gene\ppt_real\output_directory\23_14.jpg
帧发生变化: 15, 差分像素1: 16315, 差分像素2: 20957
图像已保存: D:\download\gene\ppt_real\output_directory\23_15.jpg
帧发生变化: 16, 差分像素1: 0, 差分像素2: 41441
图像已保存: D:\download\gene\ppt_real\output_directory\23_16.jpg
帧发生变化: 17, 差分像素1: 4236, 差分像素2: 35218
图像已保存: D:\download\gene\ppt_real\output_directory\23_17.jpg
帧发生变化: 18, 差分像素1: 5598, 差分像素2: 38973
图像已保存: D:\download\gene\ppt_real\output_directory\23_18.jpg
帧发生变化: 19, 差分像素1: 3736, 差分像素2: 20004
图像已保存: D:\d

帧发生变化: 37, 差分像素1: 4617, 差分像素2: 11235
图像已保存: D:\download\gene\ppt_real\output_directory\24_37.jpg
帧发生变化: 38, 差分像素1: 3779, 差分像素2: 9608
图像已保存: D:\download\gene\ppt_real\output_directory\24_38.jpg
帧发生变化: 39, 差分像素1: 2700, 差分像素2: 21697
图像已保存: D:\download\gene\ppt_real\output_directory\24_39.jpg
帧发生变化: 40, 差分像素1: 9970, 差分像素2: 38198
图像已保存: D:\download\gene\ppt_real\output_directory\24_40.jpg
帧发生变化: 41, 差分像素1: 14923, 差分像素2: 8351
图像已保存: D:\download\gene\ppt_real\output_directory\24_41.jpg
已保存PDF: D:\download\gene\ppt_real\output_directory\动植物基因表达调控(研)_于明_动植物基因表达调控(研)(第24讲)_1.pdf
正在处理文件: D:\download\gene\ppt_real\动植物基因表达调控(研)_于明_动植物基因表达调控(研)(第25讲)_1.mp4
文件名：动植物基因表达调控(研)_于明_动植物基因表达调控(研)(第25讲)_1.mp4，修改时间：2025-01-06 22:09:53.454425
帧发生变化: 1, 差分像素1: 445, 差分像素2: 8879
图像已保存: D:\download\gene\ppt_real\output_directory\25_1.jpg
帧发生变化: 2, 差分像素1: 597, 差分像素2: 9347
图像已保存: D:\download\gene\ppt_real\output_directory\25_2.jpg
帧发生变化: 3, 差分像素1: 0, 差分像素2: 9193
图像已保存: D:\download\gene\ppt_real\output_directory\25_3

图像已保存: D:\download\gene\ppt_real\output_directory\26_35.jpg
帧发生变化: 36, 差分像素1: 1590, 差分像素2: 5107
图像已保存: D:\download\gene\ppt_real\output_directory\26_36.jpg
帧发生变化: 37, 差分像素1: 2932, 差分像素2: 2517
图像已保存: D:\download\gene\ppt_real\output_directory\26_37.jpg
帧发生变化: 38, 差分像素1: 14080, 差分像素2: 11298
图像已保存: D:\download\gene\ppt_real\output_directory\26_38.jpg
帧发生变化: 39, 差分像素1: 643, 差分像素2: 6097
图像已保存: D:\download\gene\ppt_real\output_directory\26_39.jpg
帧发生变化: 40, 差分像素1: 906, 差分像素2: 4188
图像已保存: D:\download\gene\ppt_real\output_directory\26_40.jpg
帧发生变化: 41, 差分像素1: 49, 差分像素2: 5238
图像已保存: D:\download\gene\ppt_real\output_directory\26_41.jpg
帧发生变化: 42, 差分像素1: 13390, 差分像素2: 41531
图像已保存: D:\download\gene\ppt_real\output_directory\26_42.jpg
帧发生变化: 43, 差分像素1: 0, 差分像素2: 2471
图像已保存: D:\download\gene\ppt_real\output_directory\26_43.jpg
帧发生变化: 44, 差分像素1: 13570, 差分像素2: 41253
图像已保存: D:\download\gene\ppt_real\output_directory\26_44.jpg
帧发生变化: 45, 差分像素1: 0, 差分像素2: 6926
图像已保存: D:\download\gene\ppt_real\output_dire

图像已保存: D:\download\gene\ppt_real\output_directory\28_1.jpg
帧发生变化: 2, 差分像素1: 24380, 差分像素2: 36924
图像已保存: D:\download\gene\ppt_real\output_directory\28_2.jpg
帧发生变化: 3, 差分像素1: 0, 差分像素2: 6890
图像已保存: D:\download\gene\ppt_real\output_directory\28_3.jpg
帧发生变化: 4, 差分像素1: 24577, 差分像素2: 28449
图像已保存: D:\download\gene\ppt_real\output_directory\28_4.jpg
帧发生变化: 5, 差分像素1: 0, 差分像素2: 27472
图像已保存: D:\download\gene\ppt_real\output_directory\28_5.jpg
帧发生变化: 6, 差分像素1: 4310, 差分像素2: 1841
图像已保存: D:\download\gene\ppt_real\output_directory\28_6.jpg
帧发生变化: 7, 差分像素1: 4526, 差分像素2: 13906
图像已保存: D:\download\gene\ppt_real\output_directory\28_7.jpg
帧发生变化: 8, 差分像素1: 4026, 差分像素2: 14751
图像已保存: D:\download\gene\ppt_real\output_directory\28_8.jpg
帧发生变化: 9, 差分像素1: 4646, 差分像素2: 31078
图像已保存: D:\download\gene\ppt_real\output_directory\28_9.jpg
帧发生变化: 10, 差分像素1: 6581, 差分像素2: 30088
图像已保存: D:\download\gene\ppt_real\output_directory\28_10.jpg
帧发生变化: 11, 差分像素1: 5093, 差分像素2: 14334
图像已保存: D:\download\gene\ppt_real\output_directory\28_

In [ ]:
import os
from datetime import datetime
import cv2
from fpdf import FPDF
import re

# 指定视频文件所在的目录路径 (绝对路径)
directory_path = r'D:\download\gene\ppt_real'

# 指定输出图像和PDF的目录 (绝对路径)
output_directory = os.path.join(directory_path, 'output_directory')
os.makedirs(output_directory, exist_ok=True)  # 如果目录不存在，则创建

# 获取目录下所有文件的列表
file_list = os.listdir(directory_path)

# 按照文件的修改时间进行排序
file_list.sort(key=lambda x: os.path.getmtime(os.path.join(directory_path, x)))

# 遍历排序后的文件列表
for file_name in file_list:
    file_path = os.path.join(directory_path, file_name)

    # 检查是否为文件而非子目录
    if not os.path.isfile(file_path):
        continue  # 跳过非文件项

    # 获取文件的修改时间
    modified_time = datetime.fromtimestamp(os.path.getmtime(file_path))
    print(f"正在处理文件: {file_path}")
    print(f"文件名：{file_name}，修改时间：{modified_time}")

    # 打开视频文件
    cap = cv2.VideoCapture(file_path)

    # 检查视频是否成功打开
    if not cap.isOpened():
        print(f"无法打开视频文件: {file_path}")
        continue  # 跳过无法打开的视频文件

    # 初始化变量
    prev_frame_gray_1 = None
    prev_frame_gray_2 = None
    page_images = []
    i = 0

    while True:
        ret, frame = cap.read()

        if not ret:
            break  # 读取完毕或发生错误

        # 检查帧是否为空
        if frame is None:
            continue

        # 确保感兴趣区域的坐标在帧的边界内
        if frame.shape[0] < 720 or frame.shape[1] < 1280:
            print(f"帧尺寸不足，跳过该帧，帧尺寸：{frame.shape}")
            continue

        # 调整感兴趣区域的坐标以适应帧的实际尺寸
        roi_1 = frame[480:660, 1060:1200]  # 感兴趣区域1，往上移动100像素
        roi_2 = frame[280:500, 340:540]    # 感兴趣区域2，扩大一些

        # 将感兴趣区域转换为灰度图像
        roi_gray_1 = cv2.cvtColor(roi_1, cv2.COLOR_BGR2GRAY)
        roi_gray_2 = cv2.cvtColor(roi_2, cv2.COLOR_BGR2GRAY)

        # 初始化前一帧灰度图像
        if prev_frame_gray_1 is None:
            prev_frame_gray_1 = roi_gray_1.copy()
            prev_frame_gray_2 = roi_gray_2.copy()
            continue  # 跳过第一帧的差分检测

        # 计算当前帧和前一帧的差分图像
        frame_diff_1 = cv2.absdiff(roi_gray_1, prev_frame_gray_1)
        frame_diff_2 = cv2.absdiff(roi_gray_2, prev_frame_gray_2)

        # 对差分图像进行阈值处理
        _, threshold_1 = cv2.threshold(frame_diff_1, 30, 255, cv2.THRESH_BINARY)
        _, threshold_2 = cv2.threshold(frame_diff_2, 30, 255, cv2.THRESH_BINARY)

        # 统计阈值图像中非零像素的数量
        diff_pixels_1 = cv2.countNonZero(threshold_1)
        diff_pixels_2 = cv2.countNonZero(threshold_2)

        # 如果两个区域的差分像素数量都超过阈值，则认为帧发生了变化
        if (diff_pixels_1 > 100 and diff_pixels_2 > 300) or diff_pixels_1 > 150 or diff_pixels_2 > 1000:
            print(f"帧发生变化: {i + 1}, 差分像素1: {diff_pixels_1}, 差分像素2: {diff_pixels_2}")

            # 提取文件名中的编号部分
            match = re.search(r'第(\d+)讲', file_name)
            if match:
                lecture_number = match.group(1)
            else:
                lecture_number = 'unknown'

            # 生成简化的文件名
            simplified_file_name = f"{lecture_number}_{i + 1}.jpg"
            page_path = os.path.join(output_directory, simplified_file_name)

            # 尝试保存图像
            try:
                # 检查输出目录是否存在
                if not os.path.exists(output_directory):
                    os.makedirs(output_directory, exist_ok=True)
                    print(f"创建输出目录: {output_directory}")

                # 保存图像
                success = cv2.imwrite(page_path, frame)
                if success:
                    print(f"图像已保存: {page_path}")
                    page_images.append(page_path)
                else:
                    print(f"图像保存失败: {page_path}")
            except Exception as e:
                print(f"保存图像时发生错误: {str(e)}")
            i += 1

        # 更新前一帧
        prev_frame_gray_1 = roi_gray_1.copy()
        prev_frame_gray_2 = roi_gray_2.copy()

        # 显示当前帧 (可选，运行时可关闭窗口加快速度)
        # cv2.imshow("Frame", frame)

        # 跳过一定数量的帧以加快处理速度
        current_frame = cap.get(cv2.CAP_PROP_POS_FRAMES)
        max_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        # 计算目标帧的索引（例如跳500帧）
        target_frame = current_frame + 500
        if target_frame >= max_frames:
            break  # 超过总帧数，结束处理
        cap.set(cv2.CAP_PROP_POS_FRAMES, target_frame)

        # 按下 'q' 键退出循环 (可选，已注释)
        # if cv2.waitKey(1) & 0xFF == ord('q'):
        #     break

    cap.release()
    cv2.destroyAllWindows()

    # 创建PDF文件
    pdf = FPDF(orientation='L', unit='mm', format='A4')  # 使用横向A4纸
    pdf.set_auto_page_break(auto=True, margin=15)

    for image_path in page_images:
        if os.path.exists(image_path):
            pdf.add_page()
            # 计算图片的宽度和高度，以适应PDF页面
            pdf_width = pdf.w - 10  # 左右各5的边距
            pdf_height = pdf.h - 20  # 上下各10的边距
            pdf.image(image_path, x=5, y=10, w=pdf_width, h=pdf_height)
        else:
            print(f"图像文件不存在: {image_path}")

    # 保存PDF
    pdf_file_name = os.path.splitext(file_name)[0] + '.pdf'
    pdf_path = os.path.join(output_directory, pdf_file_name)
    try:
        pdf.output(pdf_path)
        print(f"已保存PDF: {pdf_path}")
    except Exception as e:
        print(f"保存PDF时发生错误: {str(e)}")